# FT-ICR analysis - PART 2

Created on 01 March 2020 for the Pentatrap experiment

@author: Menno Door<br>
@contact: door+fticr@mpi-k.de<br>
@license: MIT license
 
### Refenences :

For references reguarding the theory behind ft-icr detection and analysis methods, please check the references.md file.

### Introduction

This part of the analysis requires pre-analysed data of PART 1, meaning fitted frequencies and phase data. This part determines frequencies from phase data, calculating the free cyclotron frequency, filters/masks data and calculats the ratios using multiple analysis methods. The pre-analysed data has to be given in csv data files in one folder.

### Contents:
- step 0: loading data from PART1
- step 4: N determination (\"phase unwrap\")
- step 5: Frequency determination from phase (currently only calc nu_p from phase)
- step 6: calc nu_c (6.1 single phase sideband analysis... this we will get rid of)
- step 7: filter frequency results
- step 8: determine R using naive method
- step 9: determine R using interpolation method
- step 10: determine R using polynomial method
- step 11: determine R using cancellation-naive method
- step 12: determine R using cancellation-interpolation method
- step 13: determine R using cancellation-polynomial method
- step -1: compare results

### README & general hints

This is a short summary of steps you might want to check and problems you might run into.\n",

    0) When you start a new analysis, definitly restart the kernel of the notebook and clear the outputs. Use the \"fast-forward\"-button (two arrow-head showing right) or go to Kernel -> Restart & Clear Output
    1) Check the input parameters: Two cells below from this one there is a cell tagged \"parameters\" which lists the main variables and information needed for the analysis. This includes folders or settings for fits or other stuff. Please check it out to see what you may need to cheange there, each parameter is explained. There can be also a settings file in a measurement folder where people saved their settings (e.g. post_unwrap is set to True and its probably needed for this measurement...). These settings are always overwritten with the settings given here, so if you want to just make settings from the file, comment out the parameter in the settings dict here. 
    2) Please checkout the overview in Step 0 (loading data), how stabe is the axial frequency, the phases... Also there is an input data filter, typically given in the filter settings file (see parameter list), so if you see already in these plots that some data in the end is just random noise (phase), adjust the filter settings file and dont even load the data, that makes the analysis much easier.
    3) Check the last table in the output of Step 4 (N determination). Check if there is a wrongly determined N by checking the nu_p and end_phase. If its a q/m doublet the nu_p should be the same ish, if the phases are different by roughly 2 pi (phases are unwraped, so its possible to have 2pi), there should be a difference in N by 1. If you dont have a q/m doublet you should estimate by hand what a +-1 in N would change for your ratio and check in the end if everything checks out (compared to expected ratio).
    4) Check the filter plot in the end of Step7, maybe some values have been auto-filtered, which you might have kept or the other way around. Especially for main cycles with more false data then good, there could be problems. Filtered values are just masked, all saved files still include them.
    5) There are multiple ways to determine R from the data, all results should end up with the same value, the errors can be very different though.
    6) At the end far down, the different results will be compared.

### Requirements:

The following code was written in Python 3.7/3.8. The required libraries are listed below with a rough description for their task in the code.

    pandas (data organisation, calculation and visualization)
    numpy (calculation)
    matplotlib (plotting)
    scipy (chi square fitting)
    jupyter (Python notebook environment)
    ipywidgets (https://ipywidgets.readthedocs.io/en/latest/user_install.html)
    plotly (plotting, https://github.com/plotly/plotly.py#installation)
    qgrid (data visualization, https://github.com/quantopian/qgrid#installation)
    
### TODO:

- mean reference phase is somehow bugged, either its a trap 3 thing or something due to high value reference phases, resulting in highly negative measurement phases... dont known, somethings messud up there, but I dont get where...
- change to plotly at some of the smaller plots where it makes sense.
- step? (test): normallity test function
- step8 (polyfit): SettingWithCopyWarining ... someone cares?
- step8 (polyfit): add another error esitmation for the fit: residuals + Kolmogorow-Smirnow-Test if the residuals are Gauß-distributed

### PROBLEMS:


In [ ]:
# standard libs
from datetime import datetime, timedelta
from pathlib import Path
import os, json, time
from pprint import pprint

# math and data
import numpy as np
import scipy
from scipy import constants
import pandas as pd

# visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 300em; }</style>"))

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

try:
    %matplotlib inline
    show_tag = True
except:
    show_tag = False
    print("no ipython backend")
    
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import qgrid

py.init_notebook_mode()

plt.rcParams['figure.figsize'] = (10, 4)

# this package
%load_ext autoreload
%autoreload 2
from fticr_toolkit import *
from fticr_toolkit import ideal_trap_physics as itp

### WARNING !!! ###
# this will remove the warnings from pandas regarding assignments of copies of DataFrames to the other/same DataFrames. This
# is fine regarding the features implemented, but if you change stuff and its not working out as you think, you should 
# enable this again, to check if this is the problem.
pd.options.mode.chained_assignment = None  # default='warn'
# define floating points on display, this will be changed later to higher precision
pd.options.display.float_format = '{: .3f}'.format

In [ ]:
##   P A R A M E T E R   L I S T   ( this is especially used for batch processing using the papermill package but its also nice to have thesse parameters all at one place )

measurement_folder = "E:\\local_analysis\\Yb_CHAR\\176Yb44+_176Yb43+_176Yb44+_4"
measurement_folder = "E:\\\\local_analysis\\Yb\\172Yb43+_176Yb44+_172Yb43+"
measurement_folder = "E:\\local_analysis\\Yb168172\\172Yb43+_168Yb42+_172Yb43+_3"
measurement_folder = "Z:\\DATA_for_ANALYSIS\\analysis_Menno\\Yb174172\\174Yb42+_172Yb42+_174Yb42+_var_10"
measurement_folder = "G:\\Yb\\172174_var3_opti\\174Yb42+_172Yb42+_174Yb42+_var3_48_opti"
measurement_folder = "G:\\Yb\\172174_var1\\174Yb42+_172Yb42+_174Yb42+_var_6"
measurement_folder = "G:\\Yb\\172176_var\\176Yb42+_172Yb42+_176Yb42+_var_11"
#measurement_folder = "G:\\Yb\\172170_var\\172Yb42+_170Yb42+_172Yb42+_var_8"
measurement_folder = "G:\\Yb\\17241+42+_binding_var\\172Yb42+_172Yb41+_172Yb42+_var_1"
#measurement_folder = "G:\\Yb\\17242+43+_binding_var\\172Yb42+_172Yb43+_172Yb42+_var_3"
measurement_folder = "G:\\Yb_dualPnPtest\\172Yb43+_dual_num_nup_test_3"
measurement_folder = "G:\\Yb\\17241+42+_binding_var_dualPnP\\172Yb42+_172Yb41+_172Yb42+_dual_8"

#measurement_folder = "Z:/DATA_for_ANALYSIS/analysis_Menno/20_Re_Os_ratio_cancellation/187Re29+_187Os29+_187Re29+_12"

input_folder = "./part1_data/" # the measurements subfolder where the pre-analysed data was saved (output of PART 1: spectra fits and/or phases) 
output_folder = "./results/" # subfolder where the results of this analysis should go
# analysis parameters are loaded from a (more static and measurement specific) settings file ('analysis_settings.json') inside the measurement folder
# for testing and batch processing the parameters can be adjusted below
settings = {
    # The filter settings are used to remove some data, which is nice if e.g. the ion was lost at some point and you dont want to see crappy random data. Either directly supply a pandas dset (more usefull for papermill batches) or rather
    # create a csv file as in the examples and just assign None to this variable, it will use the csv file then. 
    "grouping": [6],
    "filter_settings": None, # pandas dset with mc, trap, position, min_cycle, max_cycle; None: try to get filter settings from loaded data; False: no filter applied
    #"filter_settings": pd.DataFrame(
    #    columns=["mcycle", "trap", "position", "min_cycle", "max_cycle"],
    #    data=[[1,2,"position_1", 0, 26],[1,2,"position_2", 0, 26],[1,3,"position_1", 0, 26],[1,3,"position_2", 0, 26]]
    #          ),
    "post_unwrap": False, # if this is True, the analysis will automatically choose the post-unwrap (the pre-unwrap of the next main cycle) for nu_p frequency determination. Maybe the whole post-unwrap part is commented out in step 5, please check.
    "unwrap_range": 6,
    "mean_ref_phase": True, # BUGGED!!!! DONT USE! if True, the reference phase will be averaged and the same value substracted for all long phases (not in Ndet), otherwise every single measured reference phase is substraced from the respective long phase
    "phase_error_undrift": True, # makes everything worse supprisingly
    "phase_filter": True, # filter measured phases by 3 sigma filter inside subcycle
    "single_axial": True, # use single spec no average axial data
    "average": False, # averaging subcycle data to match average_idx from part 1
    "nu_z_from_config": False, # default: False, if float, the config nu_z will be taken and the float value is used as the error on the config nu_z
    "fill_nu_z_from_config": False, #0.08, # default: False, if float, the config nu_z will be taken if the measured nu_z is off by hardcoded 100 Hz (which also includes no value at all) and the float value is used as the error on the config nu_z
    "sideband": False, # use sideband relation to calculate nu_c
    "nu_m2_from_theory": False, # use the magnetron freq of one position to calculate the magnetron of the other position, keeping the difference right, common offset doesn't matter
    "polydegrees": 'auto', # number of degree of the polynom fit
    "polygrouping": 'auto', # group sizes for the polynom fit
    "poly_mode": "curvefit", # routine for polynom fitting
    "poly_criterion": "AICc", # criterion for best model/poly-degree
    "invert": False,
}

### Step 0: Load data

Please supply the measurement folder and filenames to the csv datasets of pre-analysed data. 

In [ ]:
data, meas_config, results_dir = data_conversion.load_data(measurement_folder=measurement_folder,
                                              input_data=input_folder, # inside measurement folder
                                              output_folder=output_folder, # inside measurement folder
                                              measurement_script = "unwrap_n_measure_repeat") # inside measurement folder

try:
    settings = data_conversion.load_settings(measurement_folder, settings)
except:
    print("no settings from file!")
print("settings", settings)
#print("data", data.keys())
#print("invert? (default=True)", settings['invert'])

try:
    data = data_conversion.input_filter(data, settings=settings["filter_settings"])
except:
    #raise
    print("no input filter")

# define local namespace names for the data, just easier
nu_p_unwrap = data["pre_unwrap_phase"]
nu_m_unwrap = data["pre_unwrap_phase2"]
nu_p_phases = data["phase_data"]
nu_m_phases = data["phase2_data"]
axial_data = data["axial_data"]
if settings.get("single_axial", False):
    axial_data = data["axial_data_single"]

# get the variables/lists we will be looping over...
mcs = nu_p_unwrap.mcycle.unique()
print(" >>> MAIN CYCLES: ", mcs)
super_cycle_list = []
for mc in mcs:
    subset = axial_data[axial_data['mcycle']==mc]
    subcycles = subset.cycle.unique()
    super_cycle_list.append(list(subcycles))
print(" >>> SUB CYCLES: ", super_cycle_list, "\n")
positions = axial_data.position.unique()
print(positions)
start_position = meas_config["start_position"]
other_position = positions[0]
if start_position == other_position:
    try:
        other_position = positions[1]
    except:
        other_position = start_position
    
print(" >>> POSITIONS: ", positions, " - > start at", start_position)
traps = nu_p_unwrap.trap.unique()
print(" >>> TRAPS:     ", traps)

if show_tag:
    # ...and just roughly check the data...
    print(" >>> AXIAL DATA >>> ")
    #axial_data = axial_data[ (axial_data['mcycle']==1) & (axial_data['cycle'] >= 0) ]
    fig = px.scatter(axial_data, x="time", y="nu_z", error_y="dnu_z", facet_row="position", facet_col="trap", hover_data=['mcycle', 'cycle', 'position'])
    #fig = px.scatter(axial_data, x="time", y="nu_z", error_y="dnu_z", facet_col="trap", color="position", hover_data=['mcycle', 'cycle', 'position'])
    fig.update_yaxes(matches=None, showticklabels=True)
    fig.show()
    #display(axial_data)
    
    try:
        fig = px.scatter(axial_data, x="time", y="Q", error_y="dQ", facet_col="trap", color='position', hover_data=['mcycle', 'cycle', 'position'])
        fig.update_yaxes(matches=None, showticklabels=True)
        fig.show()
    except:
        pass
    
    try:
        fig = px.scatter(axial_data, x="time", y="nu_res", error_y="dnu_res", facet_col="trap", facet_row='position', hover_data=['mcycle', 'cycle', 'position'])
        fig.update_yaxes(matches=None, showticklabels=True)
        fig.show()
        for gname, grp in axial_data.groupby(["trap", "position"]):
            print(gname, statistics.complete_mean_and_error(grp.nu_res.to_numpy(), grp.dnu_res.to_numpy()), np.std(grp.nu_res.to_numpy()))
    except:
        raise
        

    try:
        axial_data["offnu_res"] = axial_data["nu_z"] - axial_data["nu_res"]
        axial_data["doffnu_res"] = np.sqrt(axial_data["dnu_z"].to_numpy()**2 + axial_data["dnu_res"].to_numpy()**2)
        fig = px.scatter(axial_data, x="time", y="offnu_res", error_y="doffnu_res", facet_col="trap", facet_row='position', hover_data=['mcycle', 'cycle', 'position'])
        fig.update_yaxes(matches=None, showticklabels=True)
        fig.show()
    except:
        pass
    
    """ Allen Deviation for axial
    for trap, data in axial_data.groupby("trap"):
        data = data.sort_values("time")
        data['epoch'] = data["time"].astype("int64")//1e9
        data['seconds'] = data['epoch'] - data['epoch'].min()
        t = data.seconds.to_numpy()/60
        y = data.nu_z.to_numpy()
        y = y/np.mean(y)
        visualization.allanvariance(y, t, plot=True)
    """

    print(" >>> PHASE DATA >>> ")
    nu_p_phases["acc_time"] = nu_p_phases["acc_time"].astype(str)
    #fig = px.scatter(nu_p_phases, x="time", y="phase", facet_row="trap", facet_col="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    fig = px.scatter(nu_p_phases, x="time", y="phase", facet_row="position", facet_col="trap", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    #fig = px.scatter(nu_p_phases, x="time", y="phase", facet_col="trap", color="position", hover_data=['mcycle', 'cycle', 'position'])
    fig.show()
    nu_p_phases["acc_time"] = nu_p_phases["acc_time"].astype(float)

    
    print(" >>> PHASE DATA >>> ")
    nu_m_phases["acc_time"] = nu_m_phases["acc_time"].astype(str)
    #fig = px.scatter(nu_p_phases, x="time", y="phase", facet_row="trap", facet_col="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    fig = px.scatter(nu_m_phases, x="time", y="phase", facet_row="position", facet_col="trap", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    #fig = px.scatter(nu_p_phases, x="time", y="phase", facet_col="trap", color="position", hover_data=['mcycle', 'cycle', 'position'])
    fig.show()
    nu_m_phases["acc_time"] = nu_m_phases["acc_time"].astype(float)
    
    print(" >>> PHASE DIFFERENCE START POSITION - OTHER POSITION >>> ")
    try:
        nu_p_phases["acc_time"] = nu_p_phases["acc_time"].astype(str)
        posA = nu_p_phases[nu_p_phases["position"] == start_position]
        posB = nu_p_phases[nu_p_phases["position"] != start_position]
        posA["phase_diff"] = posA.phase.to_numpy() - posB.phase.to_numpy()
        fig = px.scatter(posA, x="time", y="phase_diff", facet_row="trap", facet_col="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
        fig.show()
    except Exception as e:
        print(e)
        print(' DIFF only works with equal size phase data')
    finally:
        nu_p_phases["acc_time"] = nu_p_phases["acc_time"].astype(float)
        
    print(" >>> UNWRAP DATA >>> ")
    nu_p_unwrap["acc_time"] = nu_p_unwrap["acc_time"].astype(str)
    fig = px.scatter(nu_p_unwrap, x="time", y="phase", facet_col="trap", facet_row="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    fig.show()
    nu_p_unwrap["acc_time"] = nu_p_unwrap["acc_time"].astype(float)
    
    nu_m_unwrap["acc_time"] = nu_m_unwrap["acc_time"].astype(str)
    fig = px.scatter(nu_m_unwrap, x="time", y="phase", facet_col="trap", facet_row="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
    fig.show()
    nu_m_unwrap["acc_time"] = nu_m_unwrap["acc_time"].astype(float)
    
    """
    # trap2
    t2_phases = nu_p_unwrap[ (nu_p_unwrap['trap']==2) & (nu_p_unwrap['position']=='position_2') ]
    #display(t2_phases)
    accs = []
    pstds = []
        accs.append( float(group.acc_time.unique()) )
        pstds.append( np.std( np.unwrap( group.phase.to_numpy() ) ) /2/np.pi*360 )
    plt.scatter(accs, pstds, marker='o')
    plt.show()
    
    t3_phases = nu_p_unwrap[ (nu_p_unwrap['trap']==3) & (nu_p_unwrap['position']=='position_2') ]
    #display(t3_phases)
    accs = []
    pstds = []
    for name, group in t3_phases.groupby(['mcycle', 'acc_time']):
        accs.append( float(group.acc_time.unique()) )
        pstds.append( np.std( np.unwrap( group.phase.to_numpy() ) ) /2/np.pi*360 )
    plt.scatter(accs, pstds, marker='o')
    plt.show()
    """

### Step 4: N determination

The unwrap data (in this case just for the nu_p phase measurement) is used to calculate the total N of osciallations during the phase accumulation time in the later measurement

The method n_determination.fit_N will unwrap the phases for each acc_time, average, substract the reference phases from all the other measured phases and then determine the N by calculating Ns for a 1 Hz range around the guessed frequency and searches for the minimum. A plot of the Ns and the found minimum will be plotted to check the results.

The result will be a dataFrame including N, end_phase and frequency for all main cycles, traps and positiions.

In [ ]:
# each ion in each trap its own N determination, for every main cycle! (no grouping here, wouldn't make any sense)

columns = ["mcycle", "trap", "position", "N", "end_phase", "nu_p", "ion", "time", "max_acc_time"]
nu_p_N = pd.DataFrame(columns = columns)

for mc in mcs:
    for trap in traps:
        for pos in positions:
            print(mc, trap, pos)
            subset = nu_p_unwrap[(nu_p_unwrap["mcycle"] == mc) & (nu_p_unwrap["trap"] == trap) & (nu_p_unwrap["position"] == pos)]

            #subset = subset[subset["acc_time"] != subset["acc_time"].max()]
            
            # NOTE: if the structure of the config changed, you have to adjust here!
            nu_p_guess = meas_config[pos]["configuration"]["traps"][trap]["nu_p"]
            mtype = meas_config['type']
            if mtype == 'PnA':
                nphase = True
            else:
                nphase = False
            ion_str = meas_config[pos]["configuration"]["traps"][trap]["ion"]
            #evolution_time = abs(meas_config["accumulation_time"][0]["time"] - meas_config["accumulation_time"][1]["time"])

            #print(" >>> mc", mc, "trap", trap, "pos", pos, " <<< ")
            try:
                nu_range = settings.get("unwrap_range", 3)
                avg_subset = phase_analysis.prepare_unwrap_phases(subset, val="phase", show=True)
                display(avg_subset)
                N, end_phase, nu_p, mean_time, max_acc_time, Nquality = phase_analysis.determine_N(avg_subset, nu_p_guess, 
                                                                        negative=nphase, resolution=None, nu_range=nu_range, show=show_tag)
                new_row = pd.Series([mc, trap, pos, N, end_phase, nu_p, ion_str, mean_time, max_acc_time], index=nu_p_N.columns )
                print(Nquality)

                nu_p_N = nu_p_N.append(new_row, ignore_index=True)
            except:
                raise

# show results and save to csv in results folder
display(nu_p_N)
nu_p_N.to_csv(results_dir + "step4_nu_p_N.csv")
nu_p_N.to_csv(results_dir + "step4_nu_p_N.txt", sep="\t")

## NEW now also for nu_m

In [ ]:
# each ion in each trap its own N determination, for every main cycle! (no grouping here, wouldn't make any sense)

columns = ["mcycle", "trap", "position", "N", "end_phase", "nu_m", "ion", "time", "max_acc_time"]
nu_m_N = pd.DataFrame(columns = columns)

for mc in mcs:
    for trap in traps:
        for pos in positions:
            print(mc, trap, pos)
            subset = nu_m_unwrap[(nu_m_unwrap["mcycle"] == mc) & (nu_m_unwrap["trap"] == trap) & (nu_m_unwrap["position"] == pos)]

            #subset = subset[subset["acc_time"] != subset["acc_time"].max()]
            
            # NOTE: if the structure of the config changed, you have to adjust here!
            nu_m_guess = meas_config[pos]["configuration"]["traps"][trap]["nu_m"]
            mtype = meas_config['type']
            if mtype == 'PnA':
                nphase = False
            else:
                nphase = True
            ion_str = meas_config[pos]["configuration"]["traps"][trap]["ion"]
            #evolution_time = abs(meas_config["accumulation_time"][0]["time"] - meas_config["accumulation_time"][1]["time"])

            #print(" >>> mc", mc, "trap", trap, "pos", pos, " <<< ")
            try:
                nu_range = 10# settings.get("unwrap_range", 5)
                avg_subset = phase_analysis.prepare_unwrap_phases(subset, val="phase", show=True)
                display(avg_subset)
                N, end_phase, nu_m, mean_time, max_acc_time, Nquality = phase_analysis.determine_N(avg_subset, nu_m_guess, 
                                                                        negative=nphase, resolution=0.0002, nu_range=nu_range, show=show_tag)
                new_row = pd.Series([mc, trap, pos, N, end_phase, nu_m, ion_str, mean_time, max_acc_time], index=nu_m_N.columns )
                print(Nquality)

                nu_m_N = nu_m_N.append(new_row, ignore_index=True)
            except:
                raise

# show results and save to csv in results folder
display(nu_m_N)
nu_m_N.to_csv(results_dir + "step4_nu_m_N.csv")
nu_m_N.to_csv(results_dir + "step4_nu_m_N.txt", sep="\t")

In [ ]:
nu_p_N["Nplus"] = 0
backup_nu_p_N = nu_p_N.copy(deep=True)

print("1/N", 1/float(nu_p_N.tail(1).N))
print("N+1/N+1 - N/N", (float(nu_p_N.tail(2).iloc[0].N)+1)/(float(nu_p_N.tail(1).N)+1) - (float(nu_p_N.tail(2).iloc[0].N))/(float(nu_p_N.tail(1).N)))
print("N+2/N+2 - N/N", (float(nu_p_N.tail(2).iloc[0].N)+2)/(float(nu_p_N.tail(1).N)+2) - (float(nu_p_N.tail(2).iloc[0].N))/(float(nu_p_N.tail(1).N)))
print("N+2/N+1 - N/N", (float(nu_p_N.tail(2).iloc[0].N)+2)/(float(nu_p_N.tail(1).N)+1) - (float(nu_p_N.tail(2).iloc[0].N))/(float(nu_p_N.tail(1).N)))
print("2/N", 2/float(nu_p_N.tail(1).N))
print("3/N", 3/float(nu_p_N.tail(1).N))
print("18/N", 18/float(nu_p_N.tail(1).N))

print('tevol dnu(N+1)')
print(float(nu_p_N.tail(1).max_acc_time), 1/float(nu_p_N.tail(1).max_acc_time))
print(40, 1/40)
print(70, 1/70)
print(90, 1/90)
print(120, 1/120)
print(150, 1/150)
print(200, 1/200)

# check for reasonable N:
ionA, ionB = nu_p_N.ion.unique()
ion_mass_ratio = ame.get_ion_mass(ionA)[0]/ame.get_ion_mass(ionB)[0]
if ion_mass_ratio < 1: ion_mass_ratio = 1/ion_mass_ratio
print("literature ion mass ratio", ion_mass_ratio)
for gname, grp in nu_p_N.groupby(["mcycle", "trap"]):
    mc, tr = gname
    nu_pA = float(grp[grp.ion == ionA].nu_p)
    nu_pB = float(grp[grp.ion == ionB].nu_p)
    posA = grp[grp.ion == ionA].position.iloc[0]
    posB = grp[grp.ion == ionB].position.iloc[0]
    axial_grp = axial_data[(axial_data["mcycle"] == mc) & (axial_data["trap"] == tr) ]
    nu_zA = float(axial_grp[axial_grp.position == posA].nu_z.mean())
    nu_zB = float(axial_grp[axial_grp.position == posB].nu_z.mean())
    nu_mA = meas_config[posA]["configuration"]["traps"][tr]["nu_m"]
    nu_mB = meas_config[posB]["configuration"]["traps"][tr]["nu_m"]

    this_R = np.sqrt(nu_pA**2 + nu_zA**2 + nu_mA**2) / np.sqrt(nu_pB**2 + nu_zB**2 + nu_mB**2)
    if this_R < 1: this_R = 1/this_R
    print(gname, this_R, this_R - ion_mass_ratio)
    
    

# display(backup_nu_p_N)

In [ ]:
nu_p_N = backup_nu_p_N.copy(deep=True)

In [ ]:
# modify an N:
#nu_p_N.at[0, "N"] -= 10 # t2 pos 1
#nu_p_N.at[6, "N"] -= 1
#nu_p_N.at[0, "N"] += -1 # t2 pos 2
#nu_p_N.at[4, "N"] += -1
#nu_p_N.at[3, "N"] += 1 # t3 pos 1
#nu_p_N.at[7, "N"] += 1
#nu_p_N.at[2, "N"] += -1 # t3 pos 2
#nu_p_N.at[6, "N"] += -1
#nu_p_N.at[2, "N"] -= 2
display(nu_p_N)
for grpname, grp in nu_p_N.groupby(["trap", "mcycle"]):
    nup1, nup2 = grp.N.unique()
    R = nup1/nup2
    if R < 1: R = 1/R
    print(grpname, R)

### Step 5: Frequency determination from phase data

The phase data (in this case just for the nu_p phase measurement) is now evaluated to determine the corresponding frequency.

The procedure is split into to parts (two cells) and a last cell for data-handling.

#### Cell 1: Unwrapping subcycles / substract reference / unwrap main cycle / assign phase error / 3 sigma phase filter (masked)

#### Cell 2: Unwrap relative to N determination phase / Calc nu_p via phase data and N determination data (pre or post unwrap data)

#### Cell 3: The frequency determination is finished, but data has to be averaged or exanded now for the next anaylsis steps.

In [ ]:
print('nu_p phases')
display(nu_p_phases)

# just check some phase stabilities to have something to look at when your bored.
for grpname, grp in nu_p_phases.groupby(["mcycle", "trap", "position"]):
    ref_acc = grp["acc_time"].min()
    ref_phases = np.unwrap(grp[grp["acc_time"] == ref_acc].phase.to_numpy())
    print(grpname, "ref phase jitter", ref_phases.std()*180/np.pi)

print('nu_m phases')
display(nu_m_phases)
# just check some phase stabilities to have something to look at when your bored.
for grpname, grp in nu_m_phases.groupby(["mcycle", "trap", "position"]):
    ref_acc = grp["acc_time"].min()
    ref_phases = np.unwrap(grp[grp["acc_time"] == ref_acc].phase.to_numpy())
    print(grpname, "ref phase jitter", ref_phases.std()*180/np.pi)
    

In [ ]:
# unwrapping, substracting reference phase, unwrapping for full main cycle, phase filter
pd.options.display.float_format = '{: .4f}'.format

#nu_p_phases2 = nu_p_phases.copy()
nu_p_phases['dphase'] = np.nan
step5_results = pd.DataFrame()

# U N W R A P,   S U B S T R A C T   R E F E R E N C E,   U N W R A P,   P H A S E   F I L T E R
for grpname, grp in nu_p_phases.groupby(["trap", "position", "mcycle"]):
#for grpname, grp in nu_p_phases.groupby(["mcycle", "trap", "position"]):
    print(grpname)
    grp_new = filtering.three_sigma(grp, 'amp', around='median', show=False)
    grp_new = phase_analysis.unwrap_subsets(grp_new, groupby=["cycle", "acc_time"], show=False)
    grp_new = phase_analysis.substract_ref_phase(grp_new, mean=settings.get("mean_ref_phase", False), show=False)
    
    grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group="cycle", 
                                            pi_span=1, fit_N=2, fit_order=1, show=False, first_expected_offset=-np.pi/1.8, correlate=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=5, fit_order=1, show=False, skip=5)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=2, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=-3, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=-4, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=8, fit_order=3, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=-5, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=12, fit_order=5, show=False)

    # assign error to long phases
    grp_new = statistics.assign_stderr_subsets(grp_new, groupby="cycle", val="phase", dval="dphase", global_undrift=True, x="time", degree=5, show=False)
    
    step5_results = step5_results.append(grp_new, ignore_index=True)

step5_results.sort_values(by="time", inplace=True)

if show_tag:
    for grpname, grp in step5_results.groupby("trap"):
        fig, ax = plt.subplots(figsize=(15, 8))
        for subname, subgrp in grp.groupby("position"):
            t = subgrp.time
            phase = subgrp.phase /2/np.pi
            dphase = subgrp.dphase /2/np.pi
            ax.errorbar(t, phase, yerr=dphase, label=subname)
        plt.ylabel("phase (2pi)")
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:

for grpname, grp in step5_results.groupby(["trap", "mcycle"]):
    grp.sort_values(by="time", inplace=True)
    
    if not settings["post_unwrap"]:
        start_position_phase = np.median(grp[(grp["position"]==start_position) & (grp["cycle"]==grp.cycle.min())]["phase"])
        other_position_phase = np.median(grp[(grp["position"]==other_position) & (grp["cycle"]==grp.cycle.min())]["phase"])
    else:
        start_position_phase = np.median(grp[(grp["position"]==start_position) & (grp["cycle"]==grp.cycle.max()-1)]["phase"])
        other_position_phase = np.median(grp[(grp["position"]==other_position) & (grp["cycle"]==grp.cycle.max()-1)]["phase"])
    
    print(grpname, start_position_phase, other_position_phase)
    
    grp.loc[grp['position'] == start_position, 'phase'] -= start_position_phase
    grp.loc[grp['position'] == other_position, 'phase'] -= other_position_phase
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()

    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0, first_expected_offset=-np.pi/2,
    #                                     median_group='cycle', fit_N=3, fit_order=1, show=False, correlate=True, skip=2)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=-2, fit_order=1, show=False)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=2, fit_order=1, show=False)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=not settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=-2, fit_order=1, show=False)
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()
    
    grp.loc[grp['position'] == start_position, 'phase'] += start_position_phase
    grp.loc[grp['position'] == other_position, 'phase'] += other_position_phase
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()
    
    step5_results.iloc[grp.index] = grp


# check night data phase stability between 0 and 4 in the morning (silent mag-field time)
for grpname, grp in step5_results.groupby(["trap", "position"]):
    subset = grp.copy(deep=True)
    subset = subset.set_index(subset['time'], drop=True)
    subsub = subset.between_time('00:00:00', '04:00:30')
    print("mean night phase jitter trap", grpname, np.mean(subsub.dphase.unique())*180/np.pi)

#for grpname, grp in step5_results.groupby(["trap", "position"]):
#    # unwrap over complete data of this trap/position (this is safer here then in the loop before, since the next main
#    # cycles use the already stable fit from before.)
#    step5_results["phase"].loc[grp.index] = phase_analysis.drift_unwrap_method(grp["phase"].to_numpy(), pi_span = 1.3, min_fit = 1, times=grp["time"].astype('int64')//1e9)

# WARNING THIS DOES ONLY WORK IF YOU DONT HAVE RANDOM PHASES AT THE END OF THE MEASUREMENT!
# assign mean phase std (for full trap data) as phase error
for grpname, grp in step5_results.groupby(["trap", "position"]):
    mean_std = grp["dphase"].mean()
    print("mean phase error in trap", grpname, mean_std*180/np.pi)
    idx = grp.index
    step5_results["dphase"].loc[idx] = mean_std

step5_results['masked'] = False

# 3 sigma filter on phase data
if settings["phase_filter"]:
    for grpname, grp in step5_results.groupby(["mcycle", "trap", "position", "cycle"]):
        #print(grpname, grp["phase"].std()*180/np.pi)
        grp = filtering.three_sigma(grp, 'phase', err=None, undrift_xcolumn='time', manual_std=grp["dphase"].mean(), show=False)
        idx = grp.index
        step5_results["masked"].loc[idx] = grp["masked"]

step5_results.sort_values(by="time", inplace=True)

if show_tag:
    for grpname, grp in step5_results.groupby("trap"):
        fig, ax = plt.subplots(figsize=(15, 8))
        for subname, subgrp in grp.groupby("position"):
            t = subgrp.time
            phase = subgrp.phase /2/np.pi
            dphase = subgrp.dphase /2/np.pi
            ax.errorbar(t, phase, yerr=dphase, label=subname)
        plt.legend()
        plt.grid()
        plt.show()

maskedphases = step5_results[ step5_results["masked"] == True ]
display(maskedphases)


In [ ]:
# unwrapping, substracting reference phase, unwrapping for full main cycle, phase filter

#nu_p_phases2 = nu_p_phases.copy()
nu_m_phases['dphase'] = np.nan
step5_results_m = pd.DataFrame()

# U N W R A P,   S U B S T R A C T   R E F E R E N C E,   U N W R A P,   P H A S E   F I L T E R
for grpname, grp in nu_m_phases.groupby(["trap", "position", "mcycle"]):
    print(grpname)
    grp_new = filtering.three_sigma(grp, 'amp', around='median', show=False)
    grp_new = phase_analysis.unwrap_subsets(grp_new, groupby=["cycle", "acc_time"], show=False)
    grp_new = phase_analysis.substract_ref_phase(grp_new, mean=settings.get("mean_ref_phase", False), show=False)
    
    grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group="cycle", 
                                            pi_span=1, fit_N=2, fit_order=1, show=False, first_expected_offset=0, correlate=False)
    grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
                                            pi_span=1, fit_N=5, fit_order=1, show=False, skip=5)
    grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
                                            pi_span=1, fit_N=2, fit_order=1, show=False)
    grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
                                            pi_span=1, fit_N=-10, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=-4, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=8, fit_order=3, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=-5, fit_order=1, show=False)
    #grp_new = phase_analysis.grouped_unwrap(grp_new, "phase", reverse=not settings["post_unwrap"], median_group=["cycle"], 
    #                                        pi_span=1, fit_N=12, fit_order=5, show=False)

    # assign error to long phases
    grp_new = statistics.assign_stderr_subsets(grp_new, groupby="cycle", val="phase", dval="dphase", global_undrift=True, x="time", degree=5, show=False)
    
    step5_results_m = step5_results_m.append(grp_new, ignore_index=True)

step5_results_m.sort_values(by="time", inplace=True)

if show_tag:
    for grpname, grp in step5_results_m.groupby("trap"):
        fig, ax = plt.subplots(figsize=(15, 8))
        for subname, subgrp in grp.groupby("position"):
            t = subgrp.time
            phase = subgrp.phase /2/np.pi
            dphase = subgrp.dphase /2/np.pi
            ax.errorbar(t, phase, yerr=dphase, label=subname)
        plt.ylabel("phase (2pi)")
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:

for grpname, grp in step5_results_m.groupby(["trap", "mcycle"]):
    grp.sort_values(by="time", inplace=True)
    
    if not settings["post_unwrap"]:
        start_position_phase = np.median(grp[(grp["position"]==start_position) & (grp["cycle"]==grp.cycle.min())]["phase"])
        other_position_phase = np.median(grp[(grp["position"]==other_position) & (grp["cycle"]==grp.cycle.min())]["phase"])
    else:
        start_position_phase = np.median(grp[(grp["position"]==start_position) & (grp["cycle"]==grp.cycle.max()-1)]["phase"])
        other_position_phase = np.median(grp[(grp["position"]==other_position) & (grp["cycle"]==grp.cycle.max()-1)]["phase"])
    
    print(grpname, start_position_phase, other_position_phase)
    
    grp.loc[grp['position'] == start_position, 'phase'] -= start_position_phase
    grp.loc[grp['position'] == other_position, 'phase'] -= other_position_phase
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()

    grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0, first_expected_offset=0,
                                         median_group='cycle', fit_N=4, fit_order=1, show=False, correlate=True, skip=0)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=-2, fit_order=1, show=False)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=2, fit_order=1, show=False)
    #grp = phase_analysis.grouped_unwrap(grp, column='phase', reverse=not settings["post_unwrap"], pi_span=1.0,
    #                                     median_group='cycle', fit_N=-2, fit_order=1, show=False)
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()
    
    grp.loc[grp['position'] == start_position, 'phase'] += start_position_phase
    grp.loc[grp['position'] == other_position, 'phase'] += other_position_phase
    
    #plt.plot(grp["phase"].to_numpy())
    #plt.show()
    
    step5_results_m.iloc[grp.index] = grp


# check night data phase stability between 0 and 4 in the morning (silent mag-field time)
for grpname, grp in step5_results_m.groupby(["trap", "position"]):
    subset = grp.copy(deep=True)
    subset = subset.set_index(subset['time'], drop=True)
    subsub = subset.between_time('00:00:00', '04:00:30')
    print("mean night phase jitter trap", grpname, np.mean(subsub.dphase.unique())*180/np.pi)

#for grpname, grp in step5_results.groupby(["trap", "position"]):
#    # unwrap over complete data of this trap/position (this is safer here then in the loop before, since the next main
#    # cycles use the already stable fit from before.)
#    step5_results["phase"].loc[grp.index] = phase_analysis.drift_unwrap_method(grp["phase"].to_numpy(), pi_span = 1.3, min_fit = 1, times=grp["time"].astype('int64')//1e9)

# WARNING THIS DOES ONLY WORK IF YOU DONT HAVE RANDOM PHASES AT THE END OF THE MEASUREMENT!
# assign mean phase std (for full trap data) as phase error
for grpname, grp in step5_results_m.groupby(["trap", "position"]):
    mean_std = grp["dphase"].mean()
    print("mean phase error in trap", grpname, mean_std*180/np.pi)
    idx = grp.index
    step5_results_m["dphase"].loc[idx] = mean_std

step5_results_m['masked'] = False

# 3 sigma filter on phase data
if settings["phase_filter"]:
    for grpname, grp in step5_results_m.groupby(["mcycle", "trap", "position", "cycle"]):
        #print(grpname, grp["phase"].std()*180/np.pi)
        grp = filtering.three_sigma(grp, 'phase', err=None, undrift_xcolumn='time', manual_std=grp["dphase"].mean(), show=False)
        idx = grp.index
        step5_results_m["masked"].loc[idx] = grp["masked"]

step5_results_m.sort_values(by="time", inplace=True)

if show_tag:
    for grpname, grp in step5_results_m.groupby("trap"):
        fig, ax = plt.subplots(figsize=(15, 8))
        for subname, subgrp in grp.groupby("position"):
            t = subgrp.time
            phase = subgrp.phase /2/np.pi
            dphase = subgrp.dphase /2/np.pi
            ax.errorbar(t, phase, yerr=dphase, label=subname)
        plt.legend()
        plt.grid()
        plt.show()

maskedphases = step5_results_m[ step5_results_m["masked"] == True ]
display(maskedphases)


In [ ]:

# quick correlation check NU P
phases = {}
maxl = 1e6
for mc, mgrp in step5_results.groupby(["mcycle"]):
    for grpname, grp in mgrp.groupby(["trap", "position"]):
        ph = grp.phase.to_numpy()
        ph = ph-ph.min()
        t = (grp.time.astype('int64')//1e9).to_numpy()
        x = t - t.min()
        #print(x, ph)
        coef = np.polyfit(x,ph,7)
        ph -= np.poly1d(coef)(x)
        label = str(grpname[0])+str(grpname[1].split("_")[1])
        plt.plot(x, ph + 10*int(label[1]), "o", label=label)
        phases[label] = ph
        if len(ph) < maxl:
            maxl = len(ph)
    plt.title("correlation check mc" +str(mc) )
    plt.legend()
    plt.show()
    try:
        print("mcycle", mc)
        print("correlation position 1", scipy.stats.spearmanr(phases["21"][:maxl], phases["31"][:maxl]))
        print("correlation position 2", scipy.stats.spearmanr(phases["22"][:maxl], phases["32"][:maxl]))
        print("correlation trap 2", scipy.stats.spearmanr(phases["21"][:maxl], phases["22"][:maxl]))
        print("correlation trap 3", scipy.stats.spearmanr(phases["31"][:maxl], phases["32"][:maxl]))
    except:
        pass


In [ ]:

# quick correlation check NU M
phases = {}
maxl = 1e6
for mc, mgrp in step5_results_m.groupby(["mcycle"]):
    for grpname, grp in mgrp.groupby(["trap", "position"]):
        ph = grp.phase.to_numpy()
        ph = ph-ph.min()
        t = (grp.time.astype('int64')//1e9).to_numpy()
        x = t - t.min()
        #print(x, ph)
        coef = np.polyfit(x,ph,7)
        ph -= np.poly1d(coef)(x)
        label = str(grpname[0])+str(grpname[1].split("_")[1])
        plt.plot(x, ph + 10*int(label[1]), "o", label=label)
        phases[label] = ph
        if len(ph) < maxl:
            maxl = len(ph)
    plt.title("correlation check mc" +str(mc) )
    plt.legend()
    plt.show()
    try:
        print("mcycle", mc)
        print("correlation position 1", scipy.stats.spearmanr(phases["21"][:maxl], phases["31"][:maxl]))
        print("correlation position 2", scipy.stats.spearmanr(phases["22"][:maxl], phases["32"][:maxl]))
        print("correlation trap 2", scipy.stats.spearmanr(phases["21"][:maxl], phases["22"][:maxl]))
        print("correlation trap 3", scipy.stats.spearmanr(phases["31"][:maxl], phases["32"][:maxl]))
    except:
        pass


In [ ]:
qgrid_widget = qgrid.show_grid(nu_p_N)
qgrid_widget

In [ ]:
nu_p_N = qgrid_widget.get_changed_df()

In [ ]:
# calc frequency from phase using N_determination from unwrap data
step5_results['nu_p'] = np.nan
step5_results['dnu_p'] = np.nan

#settings["post_unwrap"] = not settings["post_unwrap"]
step5_results.sort_values(by="time", inplace=True)

wait = True
phaseminmax = [0, 0]
for grpname, grp in step5_results.groupby(["mcycle", "trap", "position"]):
    mc, trap, pos = grpname

    # get the results from the unwrap measurement (pre and post)
    N_data = nu_p_N[(nu_p_N['mcycle'] == mc) & (nu_p_N['trap'] == trap) & (nu_p_N['position'] == pos)]
    reverse = False

    N_data_next = nu_p_N.loc[(nu_p_N['mcycle'] == mc+1) & (nu_p_N['trap'] == trap) & (nu_p_N['position'] == pos)]
    if settings["post_unwrap"] and (len(N_data_next) != 0):
        N_data = N_data_next
        reverse = True
        print(" >>> WARNING: using Post-Unwrap! <<< ")
    else:
        print(" >>> NORMAL: using Pre-Unwrap! <<< ")
        
    #display(N_data)
    last_phase = float(N_data["end_phase"])
    last_time = int(N_data["time"].astype('int64')//1e9)
    N = float(N_data["N"])
    N += float(N_data["Nplus"])
    acc_meas = grp["acc_time"].mean()
    acc_Ndet = float(N_data["max_acc_time"])
    acc_diff = np.around(acc_Ndet - acc_meas, 3)
    print(acc_diff)
    if acc_diff != 0:
        dN = float(N_data["nu_p"])*acc_diff
        dphase = dN - np.around(dN, 0)
        print("dN", dN, np.around(dN, 0), last_phase, dphase)
        N -= np.around(dN, 0)
        last_phase -= dphase

    # now we have to unwrap all the phases in this main cycle using the last phase of the
    # N determination as a starting phase. This way we assure that the N from the N_determination
    # fits the phases in the measurement.
    #"""
    # fit phases:
    N_phases = 30
    phases = grp["phase"].to_numpy()
    times = grp["epoch"].to_numpy()
    if settings["post_unwrap"] and (len(N_data_next) != 0):
        print("post unwrap")
        fit_phases = phases[-N_phases:]
        fit_times = times[-N_phases:]
    else:
        print("pre unwrap")
        fit_phases = grp["phase"].to_numpy()[:N_phases]
        fit_times = grp["epoch"].to_numpy()[:N_phases]
    coef = np.polyfit(fit_times, fit_phases, 1)
    poly1d_fn = np.poly1d(coef)
    expected_value = poly1d_fn(last_time)
    delta = expected_value - last_phase
    counter = 0
    print("init delta", delta)
    while abs(delta) > np.pi:
        sign = delta/abs(delta)
        phases -= 2*np.pi * sign
        delta -= 2*np.pi * sign
        print("new delta", delta)
        counter += 1 * sign
        
    grp["phase"] = phases
    if True:
        plt.plot(times, phases/np.pi/2, ".", label="phases")
        plt.plot([last_time], [last_phase/np.pi/2], "o", label="N phase")
        plt.plot([last_time], [(poly1d_fn(last_time)-counter*2*np.pi)/np.pi/2], "v", label="expected N phase")
        plt.plot(times, (poly1d_fn(times)-counter*2*np.pi)/np.pi/2)
        plt.grid()
        plt.show()
    # get expected aat current phase time
    #"""
    
    #grp = phase_analysis.grouped_unwrap(grp, column="phase", start_phase_time = (last_phase, N_data["time"]), 
    #                                    reverse = reverse, fit_N=-10, fit_order=1, timesort="time", pi_span=1.0, show=False)

    """
    grp = phase_analysis.unwrap_dset(grp, column=["phase"], start_phase = last_phase, reverse = reverse,
                                     #drift_unwrap=False, drift_pi_span=1.2, show=False)
                                     #drift_unwrap="timex", drift_pi_span=1.0, timesort = True, start_phase_time = N_data["time"], show=False)
                                     drift_unwrap="timex", drift_pi_span=1.1, timesort = True, start_phase_time = N_data["time"], slope=-1.0e-3, show=False)
                                     #drift_unwrap="justfixit", drift_pi_span=1.1, timesort = True, start_phase_time = None, show=False)
                                     #drift_unwrap=False, drift_pi_span=1.3, timesort = True, start_phase_time = None, show=False)
    """
    
    # calculating frequency and error... 
    grp["nu_p"] = phase_analysis.calc_nu(N, grp["acc_time"], grp["phase"])
    grp["dnu_p"] = phase_analysis.calc_dnu(grp["acc_time"], grp["dphase"])
    print(grpname, grp["dphase"].mean(), grp["dnu_p"].mean(), grp["nu_p"].mean(), N, grp["acc_time"])
    
    step5_results.loc[grp.index] = grp


    """
    if True:        
        #plt.errorbar(grp.time, grp.phase/2/np.pi, grp.dphase/2/np.pi, marker='.', label=grpname)
        plt.errorbar(grp.epoch, grp.phase/2/np.pi, grp.dphase/2/np.pi, marker='.', label=grpname)
        #plt.plot(N_data["time"], last_phase/2/np.pi, marker='o', label=grpname)
        plt.plot(last_time, last_phase/2/np.pi, marker='o', label=grpname)
        #plt.show()
        
        #plt.plot(N_data["time"], (N+last_phase/2/np.pi)/float(N_data["max_acc_time"]), marker='o')
        if wait:
            wait = False
            #pass
        else:
            ax = plt.gca()
            ax.yaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
            ax.yaxis.set_major_locator(tck.MultipleLocator(base=1.0))

            #plt.style.use("ggplot")
            plt.grid()
            plt.legend()
            plt.show()
            wait = True
    """
"""
for grpname, grp in step5_results.groupby(["trap", "position"]):
    trap, pos = grpname
    
    N_data = nu_p_N[(nu_p_N['mcycle'] == 1) & (nu_p_N['trap'] == trap) & (nu_p_N['position'] == pos)]
    acc_max = float(N_data["max_acc_time"])
    delta_N_freq_shift = 1/acc_max/2 # this is basically 2 pi /2, so the pi step
    pi_factor = delta_N_freq_shift/(np.pi*0.5)
    print(delta_N_freq_shift, pi_factor)
    
    grp = phase_analysis.grouped_unwrap(grp, column="nu_p", fit_N=24, fit_order=3, reverse=False, timesort="time", pi_span=pi_factor, show=True)

    step5_results.loc[grp.index] = grp
""" 
#step5_results.sort_values(by='time', inplace=True)
#display(step5_results)
step5_results.to_csv(results_dir + "step5_nu_p_values.csv")
step5_results.to_csv(results_dir + "step5_nu_p_values.txt", sep="\t")
print("> masked data:")
maskedphases = step5_results[ step5_results["masked"] == True ]
display(maskedphases)

In [ ]:
fig = px.scatter(step5_results, x="time", y="nu_p", error_y="dnu_p", facet_col="trap", facet_row="position", color="masked", hover_data=['mcycle', 'cycle', 'position'])
#fig = px.scatter(step5_results, x="time", y="phase", error_y="dphase", facet_col="trap", facet_row="position", color="masked", hover_data=['mcycle', 'cycle', 'position'])
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()
display(nu_p_N)


In [ ]:
# calc frequency from phase using N_determination from unwrap data NU M
step5_results_m['nu_p'] = np.nan
step5_results_m['dnu_p'] = np.nan

#settings["post_unwrap"] = not settings["post_unwrap"]
step5_results_m.sort_values(by="time", inplace=True)

wait = True
phaseminmax = [0, 0]
for grpname, grp in step5_results_m.groupby(["mcycle", "trap", "position"]):
    mc, trap, pos = grpname

    # get the results from the unwrap measurement (pre and post)
    N_data = nu_m_N[(nu_m_N['mcycle'] == mc) & (nu_m_N['trap'] == trap) & (nu_m_N['position'] == pos)]
    reverse = False

    N_data_next = nu_m_N.loc[(nu_m_N['mcycle'] == mc+1) & (nu_m_N['trap'] == trap) & (nu_m_N['position'] == pos)]
    if settings["post_unwrap"] and (len(N_data_next) != 0):
        N_data = N_data_next
        reverse = True
        print(" >>> WARNING: using Post-Unwrap! <<< ")
    else:
        print(" >>> NORMAL: using Pre-Unwrap! <<< ")
        
    #display(N_data)
    last_phase = float(N_data["end_phase"])
    last_time = int(N_data["time"].astype('int64')//1e9)
    N = float(N_data["N"])
    N += float(N_data["Nplus"])
    acc_meas = grp["acc_time"].mean()
    acc_Ndet = float(N_data["max_acc_time"])
    acc_diff = np.around(acc_Ndet - acc_meas, 3)
    print(acc_diff)
    if acc_diff != 0:
        dN = float(N_data["nu_p"])*acc_diff
        dphase = dN - np.around(dN, 0)
        print("dN", dN, np.around(dN, 0), last_phase, dphase)
        N -= np.around(dN, 0)
        last_phase -= dphase

    # now we have to unwrap all the phases in this main cycle using the last phase of the
    # N determination as a starting phase. This way we assure that the N from the N_determination
    # fits the phases in the measurement.
    #"""
    # fit phases:
    N_phases = 30
    phases = grp["phase"].to_numpy()
    times = grp["epoch"].to_numpy()
    if settings["post_unwrap"] and (len(N_data_next) != 0):
        print("post unwrap")
        fit_phases = phases[-N_phases:]
        fit_times = times[-N_phases:]
    else:
        print("pre unwrap")
        fit_phases = grp["phase"].to_numpy()[:N_phases]
        fit_times = grp["epoch"].to_numpy()[:N_phases]
    coef = np.polyfit(fit_times, fit_phases, 1)
    poly1d_fn = np.poly1d(coef)
    expected_value = poly1d_fn(last_time)
    delta = expected_value - last_phase
    counter = 0
    print("init delta", delta)
    while abs(delta) > np.pi:
        sign = delta/abs(delta)
        phases -= 2*np.pi * sign
        delta -= 2*np.pi * sign
        print("new delta", delta)
        counter += 1 * sign
        
    grp["phase"] = phases
    if True:
        plt.plot(times, phases/np.pi/2, ".", label="phases")
        plt.plot([last_time], [last_phase/np.pi/2], "o", label="N phase")
        plt.plot([last_time], [(poly1d_fn(last_time)-counter*2*np.pi)/np.pi/2], "v", label="expected N phase")
        plt.plot(times, (poly1d_fn(times)-counter*2*np.pi)/np.pi/2)
        plt.grid()
        plt.show()
    # get expected aat current phase time
    #"""
    
    #grp = phase_analysis.grouped_unwrap(grp, column="phase", start_phase_time = (last_phase, N_data["time"]), 
    #                                    reverse = reverse, fit_N=-10, fit_order=1, timesort="time", pi_span=1.0, show=False)

    """
    grp = phase_analysis.unwrap_dset(grp, column=["phase"], start_phase = last_phase, reverse = reverse,
                                     #drift_unwrap=False, drift_pi_span=1.2, show=False)
                                     #drift_unwrap="timex", drift_pi_span=1.0, timesort = True, start_phase_time = N_data["time"], show=False)
                                     drift_unwrap="timex", drift_pi_span=1.1, timesort = True, start_phase_time = N_data["time"], slope=-1.0e-3, show=False)
                                     #drift_unwrap="justfixit", drift_pi_span=1.1, timesort = True, start_phase_time = None, show=False)
                                     #drift_unwrap=False, drift_pi_span=1.3, timesort = True, start_phase_time = None, show=False)
    """
    
    # calculating frequency and error... 
    grp["nu_p"] = phase_analysis.calc_nu(N, grp["acc_time"], grp["phase"])
    grp["dnu_p"] = phase_analysis.calc_dnu(grp["acc_time"], grp["dphase"])
    print(grpname, grp["dphase"].mean(), grp["dnu_p"].mean(), grp["nu_p"].mean(), N, grp["acc_time"])
    
    step5_results.loc[grp.index] = grp


    """
    if True:        
        #plt.errorbar(grp.time, grp.phase/2/np.pi, grp.dphase/2/np.pi, marker='.', label=grpname)
        plt.errorbar(grp.epoch, grp.phase/2/np.pi, grp.dphase/2/np.pi, marker='.', label=grpname)
        #plt.plot(N_data["time"], last_phase/2/np.pi, marker='o', label=grpname)
        plt.plot(last_time, last_phase/2/np.pi, marker='o', label=grpname)
        #plt.show()
        
        #plt.plot(N_data["time"], (N+last_phase/2/np.pi)/float(N_data["max_acc_time"]), marker='o')
        if wait:
            wait = False
            #pass
        else:
            ax = plt.gca()
            ax.yaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
            ax.yaxis.set_major_locator(tck.MultipleLocator(base=1.0))

            #plt.style.use("ggplot")
            plt.grid()
            plt.legend()
            plt.show()
            wait = True
    """
"""
for grpname, grp in step5_results.groupby(["trap", "position"]):
    trap, pos = grpname
    
    N_data = nu_p_N[(nu_p_N['mcycle'] == 1) & (nu_p_N['trap'] == trap) & (nu_p_N['position'] == pos)]
    acc_max = float(N_data["max_acc_time"])
    delta_N_freq_shift = 1/acc_max/2 # this is basically 2 pi /2, so the pi step
    pi_factor = delta_N_freq_shift/(np.pi*0.5)
    print(delta_N_freq_shift, pi_factor)
    
    grp = phase_analysis.grouped_unwrap(grp, column="nu_p", fit_N=24, fit_order=3, reverse=False, timesort="time", pi_span=pi_factor, show=True)

    step5_results.loc[grp.index] = grp
""" 
#step5_results.sort_values(by='time', inplace=True)
#display(step5_results)
step5_results.to_csv(results_dir + "step5_nu_p_values.csv")
step5_results.to_csv(results_dir + "step5_nu_p_values.txt", sep="\t")
print("> masked data:")
maskedphases = step5_results[ step5_results["masked"] == True ]
display(maskedphases)

#### Decision: Averaged or single phases?

At this point in the analysis you have to choose if you want to work with averaged data (default) or with single phases. The axial spectra typically have to be averaged to reach enough signal-to-noise-ratio to be able to fit a dip. That results in less nu_z data points than nu_p data points from phases (which we get from every subcycle measurement). If you want to work with averaged data, the phases will be averaged to match the nu_z data. If you want to work with single phases, the axial data will be assigned to all phases with an increased errorbar by np.sqrt(N). This way, averaging afterwords would result in the same axial frequency error again.

For the interpolation method, averaging is done seperatly even if you choose single phase. It just doesnt work with single phases.

In [ ]:
# merge with axial data: First just expanding the time-matching axial fits to the phase information!
step5_all_freqs = pd.DataFrame()
axial_data.sort_values(by=["mcycle", "trap", "cycle", "position"], inplace=True)
step5_results.sort_values(by=["mcycle", "trap", "cycle", "position", "time"], inplace=True)
step5_results_p = step5_results.rename(columns = {'time': 'time_p', 'phase': 'phase_p', 'dphase': 'dphase_p', 'average_idx': 'average_idx_p'})

groupby = ["mcycle", "trap", "cycle", "position"]

lastgrpname = None
average_idx = []
for grpname, grpphase in step5_results_p.groupby(groupby):
    if lastgrpname is None:
        lastgrpname = grpname
    mc, trap, cyc, pos = grpname
    #print('mcycle', mc, 'trap', trap, 'cycle', cyc, 'position', pos)
    
    grpaxial = axial_data
    for key, val in zip(groupby, grpname):
        grpaxial = grpaxial[ grpaxial[key] == val ]
    
    grpaxial_time = grpaxial.set_index('time') # set column 'time' to index
    current_axial_index = None

    subcycle = 0
    for idx, row in grpphase.iterrows():
        subcycle += 1
        # get the time-wise nearest date for axial frequencies
        try:
            axial_idx = grpaxial_time.index.get_loc(row.time_p, method='nearest')
            axial_row = grpaxial.iloc[axial_idx]
        except: continue
            
        #print('phase cycle', row['cycle'], 'axial cycle', axial_row['cycle'], 'phase time', row['time_p'], 'axial time', axial_row['time'])
        axial_row = axial_row.iloc[axial_row.index.isin(['time', 'nu_z', 'dnu_z', 'nu_res', 'dnu_res', 'Q', 'dQ', 'A', 'dA', 'off', 'doff', 'dip_width', 'ddip_width', 'fit_err', 'fit_success', 'average_idx'])]

        new_row = row.append(axial_row)
        new_row["subcycle"] = subcycle
        step5_all_freqs = step5_all_freqs.append(new_row, ignore_index=True)        
        
        # IMPORTANT! We are expanding the axial data, so we have to assign an expanded error the way that the averaged value gets the same error as before!
        if current_axial_index is None:
            current_axial_index = axial_idx
        
        if current_axial_index != axial_idx or lastgrpname != grpname:
            new_err = last_dnu_z*np.sqrt(len(average_idx))
            step5_all_freqs.loc[average_idx, "dnu_z"] = float(new_err)
            average_idx = []
            lastgrpname = grpname

        average_idx.append( step5_all_freqs.index[-1] )
        last_dnu_z = axial_row["dnu_z"]

step5_all_freqs = data_conversion.fix_column_dtypes(step5_all_freqs)

if show_tag:
    fig = px.scatter(step5_all_freqs, x="time_p", y="nu_p", error_y="dnu_p", facet_col="trap", facet_row="position", color="masked", hover_data=['mcycle', 'cycle', 'position'])
    fig.update_yaxes(matches=None, showticklabels=True)
    fig.show()

step5_all_freqs.to_csv(results_dir + "step5_all_freqs_values.csv")
step5_all_freqs.to_csv(results_dir + "step5_all_freqs_values.txt", sep="\t")

In [ ]:
# average:
if settings.get("average", False):
    new_df = pd.DataFrame()
    for grpname, grpphase in step5_all_freqs.groupby(["mcycle", "trap", "cycle", "position"]):
        avg = statistics.average_subsets(grpphase, groupby=["average_idx"], errortype="weighted",
                                              columns=["phase_p", "nu_p", "nu_z", "time_p"], 
                                              dcolumns=["dphase_p", "dnu_p", "dnu_z", None],
                                              masked=True)
        new_df = new_df.append(avg, ignore_index=True)
        cyc = grpname[2]
        if cyc == 1:
            pos = grpname[3]
            trap = grpname[1]
            nu_p = meas_config[pos]["configuration"]["traps"][trap]["nu_p"]
            nu_p_here = avg.iloc[0].nu_p
            print(trap, pos, nu_p, nu_p_here, nu_p_here - nu_p)
    
    step5_all_freqs = new_df
    for grpname, grp in step5_all_freqs.groupby(["trap", "position"]):
        print('trap, position', grpname, np.std(grp.phase_p)*180/np.pi)
        

    
    if show_tag:
        fig = px.scatter(step5_all_freqs, x="time_p", y="nu_p", error_y="dnu_p", facet_col="trap", facet_row="position", hover_data=['mcycle', 'cycle', 'position'])
        fig.update_yaxes(matches=None, showticklabels=True)
        fig.show()

    step5_all_freqs.to_csv(results_dir + "step5_all_freqs_values.csv")
    step5_all_freqs.to_csv(results_dir + "step5_all_freqs_values.txt", sep="\t")


In [ ]:
# means and compare
names = []
means = []
for grpname, grp in step5_all_freqs.groupby(["trap", "position"]):
    names.append(grpname)
    means.append(grp.nu_p.mean())

try:
    print("diffs")
    print(names[0], names[1], means[0]-means[1])
    print(names[2], names[3], means[2]-means[3])
    print(names[0], names[2], means[0]-means[2])
    print(names[1], names[3], means[1]-means[3])
    print("ratios")
    rguess = means[0]/means[1]
    print(names[0], names[1], means[0]/means[1])
    print(names[3], names[2], means[3]/means[2])
    print(names[0], names[2], means[0]/means[2])
    print(names[1], names[3], means[1]/means[3]*rguess)
except:
    pass

### Step 6: Calc free cyclotron

Use the 3 eigenfrequencies to calculate the free cyclotron frequency

In [ ]:
#step6_results = pd.DataFrame()
pd.options.mode.chained_assignment = None  # default='warn'
step6_results = step5_all_freqs.copy()
step6_results['nu_c'] = np.nan
step6_results['dnu_c'] = np.nan
step6_results['nu_m'] = np.nan
step6_results['dnu_m'] = np.nan
step6_results['nu_c_sb'] = np.nan
step6_results['dnu_c_sb'] = np.nan
step6_results['ion'] = ""

last_nu_m = {
    2: None, # ion str, nu_m
    3: None, # ion str, nu_m
    4: None, # ion str, nu_m
}

for grpname, grp in step6_results.groupby(["mcycle", "trap", "position"]):
    mc, trap, pos = grpname
    print(grpname)
    ion_str = meas_config[pos]["configuration"]["traps"][trap]["ion"]
    U0 = meas_config[pos]["traps"][trap]["U0"]
    grp["ion"] = ion_str

    # get nu_m from measurement config
    nu_m = meas_config[pos]["configuration"]["traps"][trap]["nu_m"]
    dnu_m = 0.05
    # get nu_z from measurement config
    nu_z_conf = meas_config[pos]["configuration"]["traps"][trap]["nu_z"]
    
    if settings.get("nu_m2_from_theory", False):
        B = 7#ufloat(7, 1e-6)
        if last_nu_m[trap] is None:
            last_nu_m[trap] = [ion_str, U0, nu_m]
        else:
            theo_last_omm = itp.omegam_ionstr(last_nu_m[trap][0], last_nu_m[trap][1], B=B, nominal=False, nominal_mass=True)
            theo_this_omm = itp.omegam_ionstr(ion_str, U0, B=B, nominal=False, nominal_mass=True)
            theo_delta_num = (theo_this_omm - theo_last_omm)/2/np.pi
            print("theoretical num last/this/delta", theo_last_omm/2/np.pi, theo_this_omm/2/np.pi, theo_delta_num/2/np.pi)
            print("meas num last/this/delta", last_nu_m[trap][2], nu_m, nu_m-last_nu_m[trap][2])
            old_num = nu_m
            nu_m = last_nu_m[trap][2] + theo_delta_num
            print("meas num this/new/delta", old_num, nu_m, nu_m-old_num)

    #print("nu_m", nu_m, "dnu_m", dnu_m)
    
    grp['nu_m'] = nu_m
    grp['dnu_m'] = dnu_m
    print('nu_m', nu_m, dnu_m)

    
    if settings.get("fill_nu_z_from_config", False):
        print("use config nu_z for NaNs")
        dnu_z = float( settings["fill_nu_z_from_config"] )
        grp["nu_z"] = grp["nu_z"].fillna(nu_z_conf)
        grp["dnu_z"] = grp["dnu_z"].fillna(dnu_z)
        grp=grp.replace({'dnu_z': {0: dnu_z}}) 
    
    if settings.get("nu_z_from_config", False):
        print("use config nu_z")
        nu_z = nu_z_conf
        dnu_z = float( settings["fill_nu_z_from_config"] )
    else:
        nu_z = grp["nu_z"]
        dnu_z = grp["dnu_z"]
        
    
    if pos == "position_2":
        nu_z += 0#-0.1411
    elif pos == "position_1":
        nu_z += 0#+0.1411
    
    # calc nu_c
    grp["nu_c"], grp["dnu_c"] = frequencies.calc_nu_c_error(
        grp["nu_p"], nu_z, nu_m,
        grp["dnu_p"], dnu_z, dnu_m
    )
    #print(grp["nu_c"].mean(), grp["nu_p"].mean(), grp["nu_z"].mean(), nu_m)
    
    # calc nu_c sideband
    grp["nu_c_sb"], grp["dnu_c_sb"] = frequencies.calc_nu_c_sb_error(
        grp["nu_p"], nu_m,
        grp["dnu_p"], dnu_m
    )

    # calc error impact of smaller frequencies:
    mean_nuc = grp["nu_c"].mean()
    mean_dnuc = grp["dnu_c"].mean()
    mean_nup = grp["nu_p"].mean()
    mean_dnup = grp["dnu_p"].mean()
    mean_nuz = grp["nu_z"].mean()
    mean_dnuz = grp["dnu_z"].mean()   
    mean_num = grp["nu_m"].mean()
    mean_dnum = grp["dnu_m"].mean()   
    print("nup error", mean_nup/mean_nuc*mean_dnup)
    print("nuz error", mean_nuz/mean_nuc*mean_dnuz)
    print("num error", mean_num/mean_nuc*mean_dnum)
    
    # save
    step6_results.iloc[grp.index] = grp

#display( step6_results[ (step6_results["trap"]==2) & (step6_results["position"]=="position_2") ] )
if show_tag:
    fig = visualization.compare_dset_columns(step6_results, x=["time_p", "time_p"], y=["nu_c", "nu_c_sb"], yerr=["dnu_c", "dnu_c_sb"], facet_col="trap", facet_row="position")

#display(step6_results.ion)
step6_results.to_csv(results_dir + "step6_nu_c.csv")
step6_results.to_csv(results_dir + "step6_nu_c_avg.txt", sep="\t")

### Step 7: Filter data

First we apply some automated filters, see the next cell. They should mark the most obvious outlines.
Then you should check your frequency results manually: 

The most convinient option is the filter_plot where you can choose the data you want to see and modify the "masked" property of a row in a dataset by clicking the individual point in the plot. 

Also possible to use is the filter_grid which allows you to filter columns, e.g. only show trap2, only position_1 and/or only after a certain date (TODO: time...), The modification of data is blocked except for the column "masked", where you can mark bad data.

Masking (setting "masked" = True) will not effect the nu_c calculation, but the polynomial fit method and the cancellation method.

In [ ]:
#if "filter_settings" in data:
#    filter_settings = data["filter_settings"]
#    display(filter_settings)
#    for i in range(0, len(filter_settings)):
#        row = filter_settings.iloc[i]
#        mcycle = int(row["mcycle"])
#        step6_results.at[ (step6_results["mcycle"] == mcycle) & (step6_results["cycle"] < row["min_cycle"]) , "masked"] = True
#        step6_results.at[ (step6_results["mcycle"] == mcycle) & (step6_results["cycle"] > row["max_cycle"]) , "masked"] = True#

# MANUAL PREFILTER BY CYCLES:
#step6_results.at[(step6_results["cycle"] < 14), "masked"] = True

In [ ]:
# A few automatic filtering methods TODO: could someone please check these methods? :D
pd.options.mode.chained_assignment = None  # default='warn'

if "masked" not in step6_results:
    step6_results["masked"] = False

step7_results = pd.DataFrame()
step6_results = data_conversion.fix_column_dtypes(step6_results)

for mc in mcs:
    for trap in traps:
        for pos in positions:
            # get the subset
            print(" >>> FILTERING:", "mcycle", mc, "trap", trap, "pos", pos)
            subset = step6_results[(step6_results["mcycle"] == mc) & (step6_results["trap"] == trap) & (step6_results["position"] == pos)]
            
            # filter nan values (sometimes happens when an axial spectrum is missing)
            subset = filtering.nan_filter(subset, columns=["nu_c"])

            # filter by min max boundaries for values, default: min_val: 1, max_val: 1e9:
            subset = filtering.minmax_value(subset, val="nu_z")
            subset = filtering.minmax_value(subset, val="nu_p")
            subset = filtering.minmax_value(subset, val="nu_c")
            
            # apply autofilter 3-sigma condition: calc mean of values and std
            # if value is outside of mean+-3*std, it is masked
            subset = filtering.three_sigma(subset, val="nu_z", err="dnu_z", undrift_xcolumn="time", show=False)
            subset = filtering.three_sigma(subset, val="nu_p", err="dnu_p", undrift_xcolumn="time_p", show=False)
            subset = filtering.three_sigma(subset, val="nu_c", err="dnu_c", undrift_xcolumn="time_p", show=False)
            #display(subset)

            # apply autofilter sigma-size: if sigma of value is 3 time bigger
            # then mean sigma, it is masked
            subset = filtering.sigma_size(subset, err="dnu_z")
            subset = filtering.sigma_size(subset, err="dnu_p")
            subset = filtering.sigma_size(subset, err="dnu_c")
            #display(subset)
            # save
            step7_results = step7_results.append(subset, ignore_index=True)
            
#display(step7_results)

In [ ]:
step7_results = data_conversion.fix_column_dtypes(step7_results)

if show_tag:
    fig = px.scatter(step7_results, x="time_p", y="nu_c", error_y="dnu_c", facet_col="trap", facet_row="position", color="masked", hover_data=['mcycle', 'cycle', 'position'])
    #fig = px.scatter(step7_results, x="time_p", y="nu_c", error_y="dnu_c", facet_col="trap", color="position", hover_data=['mcycle', 'cycle', 'position'])
    fig.update_yaxes(matches=None, showticklabels=True)
    fig.show()

In [ ]:
step7_results_p1 = step7_results[step7_results['position'] == 'position_1']
step7_results_p2 = step7_results[step7_results['position'] == 'position_2']

step7_results_p2['nuc_diff'] = step7_results_p1["nu_c"] - step7_results_p2["nu_c"]
step7_results_p2['dnuc_diff'] = np.sqrt(step7_results_p1["dnu_c"]**2 + step7_results_p2["dnu_c"]**2)
if show_tag:
    fig = px.scatter(step7_results_p2, x="time_p", y="nuc_diff", error_y="dnuc_diff", facet_col="trap", color="masked", hover_data=['mcycle', 'cycle', 'position'])
    #fig = px.scatter(step7_results, x="time_p", y="nu_c", error_y="dnu_c", facet_col="trap", color="position", hover_data=['mcycle', 'cycle', 'position'])
    fig.update_yaxes(matches=None, showticklabels=True)
    fig.show()

In [ ]:
# interactive plots for masking: choose the mcycle, trap, position and frequency you want to check and 
# just double click on a point to mask/unmask it. Changes are changed automatically.
step7_results['epoch_p'] = step7_results['time_p'].astype('int64')//1e9
step7_results = data_conversion.fix_column_dtypes(step7_results)
pd.options.mode.chained_assignment = None  # default='warn'

widg = "no plot"
if show_tag:
    widg = visualization.filter_plot(step7_results, groupby=["mcycle", "trap", "position"], ydata=["nu_c", "nu_z", "nu_p", "nu_c_sb"], xdata=["epoch_p"])
widg

In [ ]:
# filtering and checking the data in a table format
# be carefull!!!
# 1) if you changed "masked" values, you have to run the next cell to save them!
# 2) if you used the filter option to for example just show trap2 data or only the ones with masked==True, you have to remove these filters
#    before you save the changes with execution of the next cell, since you will only save the visible data!

qgrid_widget = visualization.filter_grid(step7_results)
qgrid_widget

In [ ]:
# NOTE: This has to run once after you changed data in the table view!!!

#changed_df = qgrid_widget.get_changed_df()
#if len(changed_df) != len(step7_results):
#    print( " >>> WARNING !!! Remove all filters in the table above and run this again.")
#else:
#    step7_results = changed_df

In [ ]:
try:
    test = step7_results[step7_results['trap']==2]
    print(test.nu_c.std(), test.dnu_c.mean(), test.dnu_c.mean()/test.nu_c.mean())
    print(test.nu_z.std(), test.dnu_z.mean(), test.dnu_z.mean()/test.nu_z.mean())

    nu_z = test.nu_z.to_numpy()
    time = test.epoch_p.to_numpy()
    time = time - time.min()
    time = time/60/60

    blubb = visualization.allanvariance(nu_z/np.mean(nu_z), time, plot=True)
except:
    print("trap 2 no data?")
    
try:
    test = step7_results[step7_results['trap']==3]
    print(test.nu_c.std(), test.dnu_c.mean(), test.dnu_c.mean()/test.nu_c.mean())
    print(test.nu_z.std(), test.dnu_z.mean(), test.dnu_z.mean()/test.nu_z.mean())

    nu_z = test.nu_z.to_numpy()
    time = test.epoch_p.to_numpy()
    time = time - time.min()
    time = time/60/60

    blubb = visualization.allanvariance(nu_z/np.mean(nu_z), time, plot=True)
except:
    print("trap 3 no data?")

In [ ]:
# NOTE: This has to run once after you changed data in the table view!!!

#changed_df = qgrid_widget.get_changed_df()
#if len(changed_df) != len(step7_results):
#    print( " >>> WARNING !!! Remove all filters in the table above and run this again.")
#else:
#    step7_results = changed_df

masked_data = step7_results[ step7_results["masked"]==True ]
display(masked_data)
step7_results.to_csv(results_dir + "step7_first_filter.csv")
step7_results.to_csv(results_dir + "step7_first_filter.txt", sep="\t")

''' For external  use...
min_nu_c = int(step7_results.nu_c.min())
min_nu_c = 16680543 + 26
min_t = step7_results.epoch_p.min()

step7_results["nu_c_min"] = min_nu_c
step7_results["nu_c_d"] = step7_results["nu_c"] - min_nu_c
step7_results["nu_c_d_mhz"] = step7_results["nu_c_d"] * 1000
step7_results["dnu_c_mhz"] = step7_results["dnu_c"] * 1000
step7_results["seconds"] = step7_results["epoch_p"] - min_t
step7_results["minutes"] = step7_results["seconds"] / 60
step7_results["hours"] = step7_results["minutes"] / 60
print(min_nu_c)
for grpname, grp in step7_results.groupby(["trap", "position"]):
    grp.to_csv(results_dir + "step7_first_filter_t"+str(grpname[0])+"_"+str(grpname[1])+".txt", sep="\t")
    sub = grp[(grp['mcycle']==1) & (grp['cycle']<10)]
    sub.to_csv(results_dir + "step7_first_filter_t"+str(grpname[0])+"_"+str(grpname[1])+"_sc1-10.txt", sep="\t")
    display(sub)
'''
pass

## NOW RATIO DETERMINATIONS!!!

#### FIRST: TRAP-WISE ANALYSIS

B: Interpolation

C: Polynomial

### Step 9: Interpolation
Data of two cycles of one position is interpolated to the time of the measurement of the in between position. Then the ratio is calculated.

In [ ]:
# For the interpolation we have to use averaged data. 
step9_data = pd.DataFrame()

if not settings["average"]:
    for grpname, grpdata in step7_results.groupby(["mcycle", "trap", "position"]):
        print("mcycle, trap, position", grpname)

        avgdata = pd.DataFrame()

        #for subname, subgrpdata in grpdata.groupby("cycle"):

        avg = statistics.average_subsets(grpdata, groupby=["cycle"], errortype="weighted",
                                              columns=["nu_c", "phase_p", "nu_p", "nu_z", "time_p"], 
                                              dcolumns=["dnu_c", "dphase_p", "dnu_p", "dnu_z", None],
                                              masked=True)
        avgdata = avgdata.append(avg)

        avgdata = data_conversion.fix_column_dtypes(avgdata)

        # filter by min max boundaries for values, default: min_val: 1, max_val: 1e9:
        avgdata = filtering.minmax_value(avgdata, val="nu_z")
        avgdata = filtering.minmax_value(avgdata, val="nu_p")
        avgdata = filtering.minmax_value(avgdata, val="nu_c")

        # apply autofilter 3-sigma condition: calc mean of values and std
        # if value is outside of mean+-3*std, it is masked
        avgdata = filtering.three_sigma(avgdata, val="nu_z", err="dnu_z", undrift_xcolumn="time", show=False)
        avgdata = filtering.three_sigma(avgdata, val="nu_p", err="dnu_p", undrift_xcolumn="time_p", show=False)
        avgdata = filtering.three_sigma(avgdata, val="nu_c", err="dnu_c", undrift_xcolumn="time_p", show=False)
        #display(subset)

        # apply autofilter sigma-size: if sigma of value is 3 time bigger
        # then mean sigma, it is masked
        avgdata = filtering.sigma_size(avgdata, err="dnu_z")
        avgdata = filtering.sigma_size(avgdata, err="dnu_p")
        avgdata = filtering.sigma_size(avgdata, err="dnu_c")

        step9_data = step9_data.append(avgdata)

    print("averaged")
else:
    step9_data = step7_results.copy(deep=True)
    
display(step9_data)

In [ ]:

non_linear_uncertainty_per_second_std = 0
step9_data = step9_data.sort_values("time_p")
step9_data.reset_index(drop=True, inplace=True)
step9_interpolated_data = pd.DataFrame()
step9_interpolate_results = pd.DataFrame()
invert = settings.get("invert", True)
ions = step9_data.ion.unique()
print(ions)

for grpname, grpdata in step9_data.groupby(["mcycle", "trap"]):
    print('mcycle, trap', grpname)
    if len(grpdata.index - sum(grpdata.index.to_numpy())) < 6:
        if len(traps)!=1:
            invert = not invert
        continue
        
    # first we try to figure out what non-linearity error we have by interpolating in only one position, so same ion, interpolating odd to even cycles
    # and determine the difference of the interpolated value to the actually measured value. The mean abs differene to the measured values corresponds
    # to a higher order fluctuation. Without any additional information this fluctuation only holds for the time step from one cycle to the next. Anyway,
    # we use it as a fluctuation per second value and then "interpolate" this fluctuation to the time step between two measurements (of different ions)
    # and use this as a non-linear-uncertainty-per-second value for the actual linear interpolation.
    non_linearities_per_second = []
    non_linearities_std_per_second = []
    mean_time_diffs = []
    for pos, posdata in grpdata.groupby("position"):
        nonlin, nonlin_std, mean_time_diff  = ratio_analysis.estimate_non_linearity_factor(posdata, y=["nu_c"], yerr=["dnu_c"], x="time_p", identifier="cycle")
        non_linearities_per_second.append(nonlin)
        non_linearities_std_per_second.append(nonlin_std)
        mean_time_diffs.append(mean_time_diff)
    
    time_diff = np.mean(mean_time_diffs)
    non_linear_uncertainty_per_second = np.mean(non_linearities_per_second)
    non_linear_uncertainty_per_second_std = np.mean(non_linearities_std_per_second)
    print("non-linearities", non_linear_uncertainty_per_second*time_diff/2, non_linear_uncertainty_per_second_std*time_diff/2)        
    print("relative non-linearities", non_linear_uncertainty_per_second*time_diff/2/grpdata["nu_c"].mean(), non_linear_uncertainty_per_second_std*time_diff/2/grpdata["nu_c"].mean())        
        
    reference, interpolate = other_position, start_position # what was measured first in time? Thats the one we want to interpolate
    print(reference, interpolate)
    
    interpolated_data = ratio_analysis.interpolate(grpdata, y=['nu_c'], yerr=['dnu_c'], groupbys=['mcycle', 'trap'], x="time_p",
                    identifier="position", id_reference=reference, id_interpolate=interpolate,
                    non_linear_uncertainty_per_second=non_linear_uncertainty_per_second_std)
    
    #display(interpolated_data.head(10))
    
    step9_interpolated_data = step9_interpolated_data.append(interpolated_data)
    
    positions = grpdata.position.unique()
    grpions = grpdata.ion.unique()
    if grpions[0] != ions[0]:
        positions = positions[::-1]
    
    #if not invert:
    #    positions = grpdata.position.unique()[::-1] 
    #else:
    #    positions = grpdata.position.unique()
    print(positions)
    
    results = ratio_analysis.calc_ratios(interpolated_data, y=['nu_c'], yerr=['dnu_c'], groupbys=['cycle'], identifier="position", ident_types=positions,
            keep_columns=['mcycle', 'trap'], additional_identifiers=['ion', 'time_p'], mean_columns=['time_p', 'time'])

    results["R"] = results["ratio_nu_c"]
    results["dR"] = results["dratio_nu_c"]
    
    #if results.R.mean() < 1:
    #    results['R'] = 1/results.R
    #    results['dR'] = results['R']**2 * results['dR']
    #    print(results.keys())
        
    step9_interpolate_results = step9_interpolate_results.append(results, ignore_index=True)
    
    #if len(step9_data.trap.unique())!=1:
    #    invert = not invert


In [ ]:
pd.options.display.float_format = '{: .12f}'.format
#step9_interpolate_results = data_conversion.fix_column_dtypes(step9_interpolate_results)
#display(step9_interpolated_data[:5])
#display(step9_interpolate_results[:5])
#display(step8_naive_results["ion_numer"].unique())

step9_interpolate_results["Rminus"] = step9_interpolate_results["R"] - 1.0

step9_interpolate_results.to_csv(results_dir + "step9_interpolate_results.csv")
step9_interpolate_results.to_csv(results_dir + "step9_interpolate_results.txt", sep="\t")

fig = px.line(step9_interpolate_results, x="time_p", y="Rminus", error_y="dratio_nu_c", facet_row='trap', hover_data=['mcycle', 'cycle', "ion_numer", "ion_denom"])
fig.update_yaxes(matches=None)
fig.show()

In [ ]:
results = {
    "interpolated ": step9_interpolate_results,
}

# R_old = 1 + 0.024 393 499 731 #+ 2e-12 # trap2

R_old = 0.024393499731
print("orig", R_old)
print(0.0243934997351 - R_old)
print(0.0243934997404 - R_old)

visualization.compare(results, groupby=["trap", "mcycle"])

### Step 10: Polynom-fit ratio determinatioin

The function ```polynom_fit.fit_sharedpoly_ratio``` does everything you need for shared polynomial fits: looping over group sizes, autogrouping, looping over polynom degrees; you can choose between different fit methods and you get all and the already filtered best settings results back; You can use it on single trap data or on trap ratio data for poly-cancel (later). It does either take single trap data or single trap ratio data. Here the docstring of the function for explanation:

In [ ]:
print(polynom_fit.fit_sharedpoly_ratio.__doc__)

If you still want to try your manual grouping, you can use the following cell to pre-group the data with the ```start_size``` and manually modify the grouping inside the plot.

In [ ]:
# you might want to do manual grouping
temp_df = pd.DataFrame()
for gname, grp in step7_results.groupby(['mcycle', 'trap']):
    grp = polynom_fit.auto_group_subset(grp, 5, 5-1)
    temp_df = temp_df.append(grp)

temp_df.sort_values(['mcycle', 'trap', 'time_p'], inplace=True)
temp_df.reset_index(drop=True, inplace=True)
#display(step7_results)

visualization.manual_grouping_plot(temp_df, x="epoch_p")

In [ ]:
step10_polyfit_data = temp_df

In [ ]:
Rguess = step9_interpolate_results['R'].mean()

step10_polyfit = pd.DataFrame()
step10_polyfit_best_fits = pd.DataFrame()
step10_polyfit_best = pd.DataFrame()
degrees = settings.get("polydegrees", 'auto')
polygrouping = settings.get("polygrouping", 'auto')
mode = settings.get("poly_mode", 'curvefit')
crit = settings.get("poly_criterion", 'AICc')
invert = settings.get("invert", True)
import time
start = time.perf_counter()

for idx, mc in enumerate(mcs):
    for idx2, trap in enumerate(traps):
        print(">>> mcycle:", mc, "trap:", trap)
        # get the subset
        subset = step10_polyfit_data[ (step10_polyfit_data["mcycle"] == mc) & (step10_polyfit_data["trap"] == trap) ]
        if len(subset.index) < 3:
            invert = not invert
            continue
        if len(subset.position.unique()) != 2:
            invert = not invert
            continue
        
        # get the testing parameters
        if isinstance(polygrouping, str) and polygrouping == 'auto':
            max_degree = int(np.ceil(len(subset.cycle.unique())/2))
            print("max degree", max_degree)
            group_sizes = list(range(3, int(max_degree+1))) # e.g. 24 data points->12 points per position->6/6 is the smallest splitting
            group_sizes.append(0) # 0 stands for full main cycle
            #group_sizes.append(-1) # -1 stands for the manual grouping
        else:
            group_sizes = polygrouping
        if not group_sizes:
            group_sizes = [3]
        print("group sizes:", group_sizes)

        if isinstance(degrees, str) and degrees == 'auto':
            max_degree = min( [ len(subset.cycle.unique())-2, 9 ] ) # maximum degree 15 ( anything else is... too much(?) )
            poly_degrees = list( range(1, max_degree) ) # e.g. full data set 24 data points - 2 for ratio and keeping one free
        else:
            poly_degrees = degrees
        if not poly_degrees:
            poly_degrees = [1]
        print("polynom degrees:", poly_degrees)

        # fit with all the settings
        subset, results, best_fits, best_results = polynom_fit.fit_sharedpoly_ratio(subset, Rguess, y="nu_c", yerr="dnu_c", data_identifier="position", 
                                               invert=invert, groupsize=group_sizes, degree=poly_degrees, mode=mode, 
                                               x="time_p", keep_columns=["mcycle", "trap"], bestfit=crit, bestgroupsize="chi2red", show=False)
        
        step10_polyfit = step10_polyfit.append(results)
        step10_polyfit_best_fits = step10_polyfit.append(best_fits)
        step10_polyfit_best = step10_polyfit_best.append(best_results)
        
        print(">>> EOA mcycle:", mc, "trap:", trap)
        
        if len(traps)!=1:
            invert = not invert
                
stop = time.perf_counter()
print("time:", stop-start)

step10_polyfit['Rminus'] = step10_polyfit['R'] - 1
step10_polyfit_best_fits['Rminus'] = step10_polyfit_best_fits['R'] - 1
step10_polyfit_best['Rminus'] = step10_polyfit_best['R'] - 1
display(step10_polyfit_best_fits)

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{: .12f}'.format
display(step10_polyfit.head(10))
display(step10_polyfit_best)

step10_polyfit.to_csv(results_dir + "step10_polyfit.csv")
step10_polyfit.to_csv(results_dir + "step10_polyfit.txt", sep="\t")
step10_polyfit_best_fits.to_csv(results_dir + "step10_polyfit_best_fits.csv")
step10_polyfit_best_fits.to_csv(results_dir + "step10_polyfit_best_fits.txt", sep="\t")
step10_polyfit_best.to_csv(results_dir + "step10_polyfit_best.csv")
step10_polyfit_best.to_csv(results_dir + "step10_polyfit_best.txt", sep="\t")

In [ ]:
# show results
polynom_fit.plot_best_results(step7_results, step10_polyfit_best, line_width = 1)


In [ ]:
#polynom_fit.compare_grouping_fits(step7_results, subset_results, groupby=['trap'], x="time_p", y="nu_c", yerr="dnu_c", ratio='R', line_width=15)

In [ ]:
#for grpname, grp in step10_polyfit.groupby(["mcycle", "trap", "groupsize"]):
    #fig = px.line(grp, x="degree", y="R", error_y="dR", color="group", hover_data=['group'], title="mcycle, trap: "+str(grpname) )
    #fig.show()
    
Rmean = pd.DataFrame()
for grpname, grp in step10_polyfit.groupby(["mcycle", "trap"]):    
    R, inner, outer, chi2red = statistics.complete_mean_and_error(grp.R.to_numpy(), grp.dR.to_numpy())
    dR = np.nanmax([inner, outer])
    print(grpname, R, inner, outer, chi2red)
    Rmean = Rmean.append( pd.Series(data=[grpname[0], grpname[1], R, dR, inner, outer, chi2red], 
                                    index=["mcycle", "trap", "R", "dR", "inner", "outer", "chi2red"]), 
                          ignore_index=True )
    
results = pd.DataFrame()
for grpname, grp in step10_polyfit.groupby(["mcycle", "trap", "groupsize", "degree"]):
    R, inner, outer, chi2red = statistics.complete_mean_and_error(grp.R.to_numpy(), grp.dR.to_numpy())
    dR = max([inner, outer])
    #print(R-1, inner, outer, chi2red)
    Rall = float(Rmean[ (Rmean["mcycle"]==grpname[0]) & (Rmean["trap"]==grpname[1]) ]["R"])
    results = results.append( pd.Series(data=[grpname[0], grpname[1], grpname[2], grpname[3], R, R-Rall, dR, inner, outer, chi2red], 
                                        index=["mcycle", "trap", "groupsize", "degree", "R", "R-Rmean", "dR", "inner", "outer", "chi2red"]), 
                              ignore_index=True )

fig = px.line(results, x="degree", y="R-Rmean", error_y="dR", color="groupsize", facet_row="mcycle", facet_col="trap" )
fig.update_layout( yaxis = dict( showexponent = 'all', exponentformat = 'e' ) )
fig.show()
fig = px.line(results, x="groupsize", y="R-Rmean", error_y="dR", color="degree", facet_row="mcycle", facet_col="trap" )
fig.update_layout( yaxis = dict( showexponent = 'all', exponentformat = 'e' ) )
fig.show()


In [ ]:
results = pd.DataFrame()
step10_polyfit_best_fits2 = pd.DataFrame(columns=step10_polyfit_best_fits.columns)
for grpname, grp in step10_polyfit.groupby(["mcycle", "trap", "groupsize", "group"]):
    mincrit = grp[crit].min()
    sub = grp[ grp[crit]==mincrit ]
    step10_polyfit_best_fits2 = step10_polyfit_best_fits2.append(sub, ignore_index=True)

print(len(step10_polyfit_best_fits2))
for grpname, grp in step10_polyfit_best_fits2.groupby(["mcycle", "trap", "groupsize"]):
    print('mc, trap, size, indexes, total', grpname[0], grpname[1], grpname[2], grp.group.unique(), len(grp), grp.degree.unique())
    #display(grp)
    R, inner, outer, chi2red = statistics.complete_mean_and_error(grp.R.to_numpy(), grp.dR.to_numpy())
    dR = max([inner, outer])
    mean_degree = float(grp.degree.mean())
    #print(R-1, inner, outer, chi2red)
    Rall = float(Rmean[ (Rmean["mcycle"]==grpname[0]) & (Rmean["trap"]==grpname[1]) ]["R"])
    results = results.append( pd.Series(data=[grpname[0], grpname[1], grpname[2], R, R-1, R-Rall, dR, inner, outer, chi2red, mean_degree], 
                                        index=["mcycle", "trap", "groupsize", "R", "Rminus", "R-Rmean", "dR", "inner", "outer", "chi2red", "mean_degree"]), 
                              ignore_index=True )
    
fig = px.line(results, x="groupsize", y="R-Rmean", error_y="dR", facet_row="mcycle", facet_col="trap", color="mean_degree", hover_data=['mcycle', "R-Rmean", "dR", "mean_degree"] )
fig.update_layout( yaxis = dict( showexponent = 'all', exponentformat = 'e' ) )
fig.show()

results.to_csv(results_dir + "step10_polyfit_grouping_best_for_plot.csv")
results.to_csv(results_dir + "step10_polyfit_grouping_best_for_plot.txt", sep="\t")
step10_polyfit_best_fits2.to_csv(results_dir + "step10_polyfit_best_fits.csv")
step10_polyfit_best_fits2.to_csv(results_dir + "step10_polyfit_best_fits.txt", sep="\t")

In [ ]:
display(step10_polyfit_best)
chi2s = step10_polyfit_best.chi2red.to_numpy()
print(chi2s)
print(chi2s.mean(), chi2s.std())

In [ ]:
results = {
    #"ame          ": one_df,
    "interpolated": step9_interpolate_results,
    "poly        ": step10_polyfit_best,
}

visualization.compare(results)

In [ ]:
print(step10_polyfit_best.columns)
display(step10_polyfit_best_fits.AICc)
print(step10_polyfit_best.chi2red)

#### SECOND: CANCELLATION ANALYSIS

B: Interpolation

C: Polynomial

##### FIRST: Data preparation (creating trap ratios) --- ONLY WITH 2 TRAP DATA!!! 
e.g.

Rpos1 = nu_c(trap2, t1) / nu_c(trap3, t1) = qmA * B(trap2, t1) / qmB / B(trap3, t1) = qmA/qmB * rohB(t1)

Rpos2 = nu_c(trap2, t2) / nu_c(trap3, t2) = qmB * B(trap2, t2) / qmA / B(trap3, t2) = qmB/qmA * rohB(t2)

for naive calculation then:

Rcancel = sqrt( Rpos1 / Rpos2 ) = sqrt( qmA^2/qmB^2 ) , if rohB(t1) == rohB(t2)

In [ ]:
step11_trap_ratio_data = pd.DataFrame()
step7_results.sort_values(["mcycle", "trap", "time_p"], inplace=True)
step7_results.reset_index(drop=True, inplace=True)
traps = [2,3] # in this position trap 2 is numerator (trap2/trap3)

unique_traps = step7_results.trap.unique()
if len(unique_traps) < 2:
    raise KeyError('not enough trap data for cancellation, only traps: '+str(unique_traps))

# ONLY WITH 2 TRAP DATA!!!
for name, grp in step7_results.groupby(["mcycle", "position"]):
    print("mcycle, trap:", name, "positions", grp.position.unique(), "subcycles", grp.subcycle.unique())

    results = ratio_analysis.calc_ratios(grp, y=['nu_c'], yerr=['dnu_c'], groupbys=['cycle', 'subcycle'], identifier="trap", ident_types=traps,
            keep_columns=['mcycle', 'position', 'time_p', 'trap'], additional_identifiers=['ion', 'time_p', 'time'], mean_columns=[])
        
    step11_trap_ratio_data = step11_trap_ratio_data.append(results, ignore_index=True)

step11_trap_ratio_data['masked'] = False
display(step11_trap_ratio_data)

In [ ]:
step11_trap_ratio_data["Rminus"] = step11_trap_ratio_data["ratio_nu_c"] - 1.0

step11_trap_ratio_data.to_csv(results_dir + "step11_trap_ratio_data.csv")
step11_trap_ratio_data.to_csv(results_dir + "step11_trap_ratio_data.txt", sep="\t")

fig = px.line(step11_trap_ratio_data, x="time_p", y="Rminus", error_y="dratio_nu_c", facet_row='position', hover_data=['mcycle', 'cycle'])
fig.update_yaxes(matches=None, showticklabels=True)
fig.show()

### Step 12: Interpolated Cancellation
The trap ratios are interpolated to the same time. It works exacly the same as the interpolation in step 9.

In [ ]:
# For the interpolation we have to have averaged data.
step12_cancel_interpol_results = pd.DataFrame()
step12_avg_data = pd.DataFrame()
for grpname, grpdata in step11_trap_ratio_data.groupby(["mcycle", "position"]):
    print('mcycle, position', grpname)
    avgdata = pd.DataFrame()
    for subname, subgrpdata in grpdata.groupby("cycle"):
        avg = statistics.average_subsets(subgrpdata, groupby=["cycle"], errortype="weighted",
                                              columns=["ratio_nu_c", "time_p"], 
                                              dcolumns=["dratio_nu_c", None],
                                              masked=False)
        avgdata = avgdata.append(avg)
        
    avgdata['masked'] = False
    
    # filter by min max boundaries for values, default: min_val: 1, max_val: 1e9:
    avgdata = filtering.minmax_value(avgdata, val="ratio_nu_c", min_val=0, max_val=100)

    # apply autofilter 3-sigma condition: calc mean of values and std
    # if value is outside of mean+-3*std, it is masked
    avgdata = filtering.three_sigma(avgdata, val="ratio_nu_c", err="dratio_nu_c", undrift_xcolumn="time_p", show=False)
    #display(subset)

    # apply autofilter sigma-size: if sigma of value is 3 time bigger
    # then mean sigma, it is masked
    avgdata = filtering.sigma_size(avgdata, err="dratio_nu_c")
       
    step12_avg_data = step12_avg_data.append(avgdata)

print("averaged")
#display(step12_avg_data)
step12_avg_data = step12_avg_data.sort_values("time_p")
step12_avg_data.reset_index(drop=True, inplace=True)
step12_interpolated_data = pd.DataFrame()
step12_cancel_interpol_results = pd.DataFrame()
positions = ["position_1","position_2"] # in this position trap 2 is numerator (trap2/trap3)
if not settings.get("invert", True):
    positions = positions[::-1]

for grpname, grpdata in step12_avg_data.groupby(["mcycle"]):
    print('mcycle', grpname)
    reference, interpolate = 'position_1', 'position_2' # what was measured first in time? Thats the one we want to interpolate

    interpolated_data = ratio_analysis.interpolate(grpdata, y=['ratio_nu_c'], yerr=['dratio_nu_c'], groupbys=['mcycle'], x="time_p",
                    identifier="position", id_reference=reference, id_interpolate=interpolate,
                    non_linear_uncertainty_per_second=0)
        
    step12_interpolated_data = step12_interpolated_data.append(interpolated_data)
    
    #if not invert:
    #    positions = ["position_1","position_2"]
    #else:
    #    positions = ["position_2","position_1"]
    
    results = ratio_analysis.calc_ratios(interpolated_data, y=['ratio_nu_c'], yerr=['dratio_nu_c'], groupbys=['cycle', 'subcycle'], identifier="position", ident_types=positions,
            keep_columns=['mcycle', 'time_p', 'trap'], additional_identifiers=['ion_numer', 'ion_denom', 'time_p'], mean_columns=[])

    step12_cancel_interpol_results = step12_cancel_interpol_results.append(results, ignore_index=True)
    

In [ ]:
#display(step11_cancel_naive_results)

# the 'ratio' calculated in the step before is acutally qmA^2 / qmB^2, so we have to take the sqrt (make sure you get the Error right!)
step12_cancel_interpol_results["R"] = np.sqrt( step12_cancel_interpol_results["ratio_ratio_nu_c"] )
step12_cancel_interpol_results["dR"] = step12_cancel_interpol_results["dratio_ratio_nu_c"]/step12_cancel_interpol_results["R"]/2
step12_cancel_interpol_results["Rminus"] = step12_cancel_interpol_results["R"] - 1    

step12_cancel_interpol_results.to_csv(results_dir + "step12_cancel_interpol_results.csv")
step12_cancel_interpol_results.to_csv(results_dir + "step12_cancel_interpol_results.txt", sep="\t")

fig = px.line(step12_cancel_interpol_results, x="time_p", y="Rminus", error_y="dR", hover_data=['mcycle', 'cycle'])
fig.show()

# TIP: it should be ion_numer_numer = ion_denom_denom and ion_numer_denom = ion_denom_numer (ending up ionA^2 / ionB^2)

In [ ]:
results = {
    "interpolated ": step9_interpolate_results,
    "poly all     ": step10_polyfit_best,
    "cancel interp": step12_cancel_interpol_results,
}

visualization.compare(results, groupby=["mcycle", "trap"])

### Step 13: Polyfit Cancellation


In [ ]:
# you might want to do manual grouping
temp_df = pd.DataFrame()
step11_trap_ratio_data['masked'] = False # the ratio calc was already masked selective.
for gname, grp in step11_trap_ratio_data.groupby(['mcycle']):
    grp = polynom_fit.auto_group_subset(grp, 4, 4-1, sortby=["cycle", "time_p"])
    temp_df = temp_df.append(grp)

temp_df.sort_values(['mcycle', 'time_p'], inplace=True)
temp_df.reset_index(drop=True, inplace=True)
#display(step7_results)

visualization.manual_grouping_plot(temp_df, groupby=["mcycle"], sets="position", y='ratio_nu_c', yerr='dratio_nu_c', x="time_p")

In [ ]:
step11_trap_ratio_data = temp_df

In [ ]:
Rguess = step9_interpolate_results['R'].mean()

step13_cancel_polyfit = pd.DataFrame()
step13_cancel_polyfit_best_fits = pd.DataFrame()
step13_cancel_polyfit_best = pd.DataFrame()
step11_trap_ratio_data['ion'] = step11_trap_ratio_data['ion_numer']
degrees = settings.get("polydegrees", 'auto')
polygrouping = settings.get("polygrouping", 'auto')
invert = settings.get("invert", True)
import time
start = time.perf_counter()

for idx, mc in enumerate(mcs):
    print(">>> mcycle:", mc)
    # get the subset
    subset = step11_trap_ratio_data[ (step11_trap_ratio_data["mcycle"] == mc) ]
    if len(subset.index) < 3:
        continue
    
    # get the testing parameters
    if isinstance(polygrouping, str) and polygrouping == 'auto':
        group_sizes = list(range(3, int(np.ceil(len(subset.cycle.unique())/4))+1)) # e.g. 24 data points->12 points per position->6/6 is the smallest splitting
        group_sizes.append(0) # 0 stands for full main cycle
    else:
        group_sizes = polygrouping
    if not group_sizes:
        group_sizes = [3]
    print("group sizes:", group_sizes)        

    if isinstance(degrees, str) and degrees == 'auto':
        max_degree = min( [ len(subset.cycle.unique())-2, 15 ] ) # maximum degree 15 ( anything else is... too much(?) )
        poly_degrees = list( range(1, max_degree) ) # e.g. full data set 24 data points - 2 for ratio and keeping one free
    else:
        poly_degrees = degrees
    if not poly_degrees:
        poly_degrees = [1]
    print("polynom degrees:", poly_degrees)

    # fit with all the settings
    subset, results, best_fits, best_results = polynom_fit.fit_sharedpoly_ratio(subset, Rguess, y="ratio_nu_c", yerr="dratio_nu_c", data_identifier="position", 
                                           invert=invert, groupsize=group_sizes, degree=poly_degrees, mode='curvefit', 
                                           x="time_p", keep_columns=["mcycle", "trap"], bestfit="AICc", bestgroupsize="chi2red", show=False)

    step13_cancel_polyfit = step13_cancel_polyfit.append(results, ignore_index=True)
    step13_cancel_polyfit_best_fits = step13_cancel_polyfit_best_fits.append(best_fits, ignore_index=True)
    step13_cancel_polyfit_best = step13_cancel_polyfit_best.append(best_results, ignore_index=True)

step13_cancel_polyfit["Rfit"] = step13_cancel_polyfit["R"]
step13_cancel_polyfit_best_fits["Rfit"] = step13_cancel_polyfit_best_fits["R"]
step13_cancel_polyfit_best["Rfit"] = step13_cancel_polyfit_best["R"]
step13_cancel_polyfit["dRfit"] = step13_cancel_polyfit["dR"]
step13_cancel_polyfit_best_fits["dRfit"] = step13_cancel_polyfit_best_fits["dR"]
step13_cancel_polyfit_best["dRfit"] = step13_cancel_polyfit_best["dR"]

stop = time.perf_counter()
print("time:", stop-start)
        
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{: .12f}'.format
display(step13_cancel_polyfit.head(10))
display(step13_cancel_polyfit_best)

In [ ]:
# show results

polynom_fit.plot_best_results(step11_trap_ratio_data, step13_cancel_polyfit_best, groupby=['mcycle'], x='time_p', y='ratio_nu_c', yerr='dratio_nu_c')

In [ ]:

# the 'ratio' calculated in the step before is acutally qmA^2 / qmB^2, so we have to take the sqrt (make sure you get the Error right!)
step13_cancel_polyfit["R"] = np.sqrt( step13_cancel_polyfit["Rfit"] )
step13_cancel_polyfit["dR"] = step13_cancel_polyfit["dRfit"]/step13_cancel_polyfit["R"]/2
step13_cancel_polyfit["Rminus"] = step13_cancel_polyfit["R"] - 1
step13_cancel_polyfit_best_fits["R"] = np.sqrt( step13_cancel_polyfit_best_fits["Rfit"] )
step13_cancel_polyfit_best_fits["dR"] = step13_cancel_polyfit_best_fits["dRfit"]/step13_cancel_polyfit_best_fits["R"]/2
step13_cancel_polyfit_best_fits["Rminus"] = step13_cancel_polyfit_best_fits["R"] - 1
step13_cancel_polyfit_best["R"] = np.sqrt( step13_cancel_polyfit_best["Rfit"] )
step13_cancel_polyfit_best["dR"] = step13_cancel_polyfit_best["dRfit"]/step13_cancel_polyfit_best["R"]/2
step13_cancel_polyfit_best["Rminus"] = step13_cancel_polyfit_best["R"] - 1

display(step13_cancel_polyfit_best)


step13_cancel_polyfit.to_csv(results_dir + "step13_cancel_polyfit.csv")
step13_cancel_polyfit.to_csv(results_dir + "step13_cancel_polyfit.txt", sep="\t")
step13_cancel_polyfit_best_fits.to_csv(results_dir + "step13_cancel_polyfit_best_fits.csv")
step13_cancel_polyfit_best_fits.to_csv(results_dir + "step13_cancel_polyfit_best_fits.txt", sep="\t")
step13_cancel_polyfit_best.to_csv(results_dir + "step13_cancel_polyfit_best.csv")

step13_cancel_polyfit_best.to_csv(results_dir + "step13_cancel_polyfit_best.txt", sep="\t")

fig = px.line(step13_cancel_polyfit_best, x="time_p", y="Rminus", error_y="dR", hover_data=['mcycle', 'cycle_start'])
fig.show()

# TIP: it should be ion_numer_numer = ion_denom_denom and ion_numer_denom = ion_denom_numer (ending up ionA^2 / ionB^2)

In [ ]:


results = {
    "interpolated": step9_interpolate_results,
    "poly": step10_polyfit_best,
    "cancel interp": step12_cancel_interpol_results,
    "cancel poly": step13_cancel_polyfit_best,
}

visualization.compare(results, timecol='time_p', groupby=["mcycle", "trap"])
visualization.allancompare(results, time='time_p')

In [ ]:
visualization.compare(results, timecol='time_p', groupby=None)

#### """
data_to_look_at = step8_naive_results_merged
#data_to_look_at = step11_cancel_naive_results_merged
#data_to_look_at = step7_results[ (step7_results['trap']==2) & (step7_results['position']=='position_2') & (step7_results['masked']==False) ]
#ycol = 'R'
ycol = 'nu_c'
tcol = 'time_p'
data_to_look_at.sort_values(tcol)

data_to_look_at['epoch'] = data_to_look_at[tcol].astype("int64")//1e9
data_to_look_at['seconds'] = data_to_look_at['epoch'] - data_to_look_at['epoch'].min()
statistics.allantest(data_to_look_at[ycol].to_numpy(), data_to_look_at.seconds.to_numpy())
"""

In [ ]:
# Some more or less interesting data checks

In [ ]:
# linear fits of axial reference phase
copyphases = nu_p_phases.copy()
copyphases['epoch'] = copyphases['time'].astype('int64')//1e9

slopes = {
    2: [],
    3: []
}
for grpname, grp in copyphases.groupby(['trap', 'position', 'acc_time']):
    if grpname[-1] != copyphases.acc_time.min():
        continue
    grp_new = phase_analysis.unwrap_dset(grp, column="phase", drift_unwrap=True, drift_pi_span=1.5, timesort=True, reverse=False, show=False)
    phases = grp_new['phase'].to_numpy()
    times = grp_new['epoch'].to_numpy()
    
    opt = np.polyfit(times,phases, 1)
    print(opt)
    slopes[grpname[0]].append(opt[0])
    plt.plot(times, phases)
    plt.plot(times, opt[1] + opt[0]*times)
    plt.show()
    
print(slopes)
print(np.mean(slopes[2])/np.mean(slopes[3]))
print(732536.1/501554.07)
print((np.mean(slopes[2])/np.mean(slopes[3]) - 732536.1/501554.07)/(732536.1/501554.07))
print((np.mean(slopes[2])/np.mean(slopes[3]) - 732536.1/501554.07))
print(724000/475000)

In [ ]:
"""
test = step9_interpolate_results[step9_interpolate_results['trap']==3]
print(test.R.std(), test.dR.mean(), test.dR.mean()/test.R.mean())

R = test.R.to_numpy()
time = test.time_p.astype("int64")//1e9
time = time.to_numpy()
time = time - time.min()
time = time/60/60

blubb = visualization.allanvariance(R/np.mean(R), time, plot=True)
"""

In [ ]:
"""
ion_num = step8_naive_results_merged['ion_numer'].unique()[0]
ion_den = step8_naive_results_merged['ion_denom'].unique()[0]
from uncertainties import ufloat

results = {
    "interpolated ": step9_interpolate_results,
    "poly all     ": step10_polyfit_best,
}

omegac_ref = omegac_ioi = omegap_ref = omegap_ioi = omegam_ref = omegam_ioi = None
exc_radius = ufloat(20, 5)*1e-6

x = []
y = []
dy = []
for trap, grp in step7_results.groupby(['trap']):

    pos_ref = grp[grp['ion'] == ion_den]
    pos_ioi = grp[grp['ion'] == ion_num]
        
    omegac_ref = ufloat(pos_ref.nu_c.mean()*2*np.pi, 0.1)
    omegap_ref = ufloat(pos_ref.nu_p.mean()*2*np.pi, 1)
    omegam_ref = ufloat(pos_ref.nu_m.mean()*2*np.pi, 1)
            
    omegac_ioi = ufloat(pos_ioi.nu_c.mean()*2*np.pi, 0.1)
    omegap_ioi = ufloat(pos_ioi.nu_p.mean()*2*np.pi, 1)
    omegam_ioi = ufloat(pos_ioi.nu_m.mean()*2*np.pi, 1)
    

    for name, data in results.items():
        
        tdata = data[data['trap'] == trap]
        ratios = tdata.R.to_numpy()
        dratios = tdata.dR.to_numpy()
        R, err_in, err_out, chi2red = statistics.complete_mean_and_error(ratios, dvalue = dratios)
        dR = max(err_in, err_out)
        uR = ufloat(R, dR)
        #print((uR-1)*1e12)
        if uR.n > 1:
            uR = 1/uR
        
        Eb_ref = ufloat(31529.1, 8.8)
        Eb_ioi = Eb_ref
        eV, Q, nRatio = systematics.neutral_mass(uR.n, uR.s, ion_den, ion_num, Rcompare=None,
                        Eb_ref=Eb_ref, Eb_ioi=Eb_ioi,
                        sys_on=True, exc_radius=exc_radius, show=False,
                         omegac_ref=omegac_ref, omegac_ioi=omegac_ioi, omegap_ref=omegap_ref, omegap_ioi=omegap_ioi,
                         omegam_ref=omegam_ref, omegam_ioi=omegam_ioi)
        
        #print((nRatio-1)*1e12)

        print(name, trap, eV, Q, nRatio, 1/nRatio, nRatio.s/nRatio.n, uR, 1/uR, uR.s/uR.n)
        x.append(name.strip()+'_'+str(trap))
        y.append(nRatio.n)
        dy.append(nRatio.s)

uRn_rana = ybrana.make_ybratio(ion_num[:-3], ion_den[:-3])
uRn_rana = 1/uRn_rana
print(uRn_rana)
plt.rcParams["figure.figsize"] = (20,10)

plt.errorbar(['rana'], [uRn_rana.n], [uRn_rana.s] )
plt.errorbar(x, y, dy)
plt.show()
#uR = 1/uR
"""